In [60]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from collections import Counter
fichier_csv = r"C:\Users\matin\Downloads\sncb_data_challenge.csv"
df = pd.read_csv(fichier_csv,sep=";")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
df.columns.tolist()
import pandas as pd
import ast
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score


# Main columns of the dataset

In [61]:
# We keep only the main column
columns_to_keep = ["incident_id", "vehicles_sequence", "events_sequence",
                    "seconds_to_incident_sequence", "incident_type", "approx_lat", "approx_lon", "train_kph_sequence", "dj_ac_state_sequence",
                    "dj_dc_state_sequence"]
df_paper = df.loc[:, columns_to_keep]
df_paper.columns.tolist()

['incident_id',
 'vehicles_sequence',
 'events_sequence',
 'seconds_to_incident_sequence',
 'incident_type',
 'approx_lat',
 'approx_lon',
 'train_kph_sequence',
 'dj_ac_state_sequence',
 'dj_dc_state_sequence']

# WE show some characteristics of the dataframe

In [3]:
incident_id = df_paper["incident_id"]
incident_id = incident_id.head(10)
incident_id
approx_lat = df_paper['approx_lat']
approx_long = df_paper["approx_lon"]
train_kph_sequence = df_paper["train_kph_sequence"]
dj_ac_state_sequence = df_paper["dj_ac_state_sequence"]
dj_dc_state_sequence = df_paper["dj_dc_state_sequence"]

In [58]:
vehicles_sequence = df_paper["vehicles_sequence"]
vehicles_sequence = vehicles_sequence.head(1)
vehicles_sequence

0    [609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609, 609,

In [57]:
events_sequence = df_paper["events_sequence"]
events_sequence = events_sequence.head(2)
events_sequence
events_sequence_raw = (df['events_sequence']).apply(ast.literal_eval).tolist()
#print(events_sequence_raw)

In [53]:
seconds_to_incident_sequence = df_paper["seconds_to_incident_sequence"]
seconds_to_incident_sequence.head(5)
seconds_to_incident_sequence

0                                                                                                           [-5510, -5510, -5507, -5507, -5506, -5506, -5506, -5505, -5505, -5504, -5504, -5504, -5503, -5494, -5472, -5469, -5460, -5458, -5311, -5311, -5056, -5046, -5045, -5044, -5044, -5044, -5043, -4229, -4229, -4229, -4226, -4181, -4159, -3975, -3975, -3973, -3973, -3971, -3970, -3969, -3969, -3945, -3944, -3942, -3940, -3937, -3888, -3885, -3785, -3783, -3779, -3714, -3713, -3713, -3702, -3640, -3639, -3583, -3546, -3546, -3546, -3542, -3478, -3473, -3471, -3471, -3471, -3437, -3428, -3422, -3413, -3407, -3399, -3361, -3353, -3341, -3332, -3308, -3135, -3135, -3132, -3132, -3131, -3131, -3129, -3129, -3102, -3099, -3099, -3098, -3097, -3083, -3081, -3026, -2983, -2981, -2981, -2944, -2907, -2897, ...]
1                                                                                                           [-8573, -8573, -8032, -8032, -8032, -7859, -6146, -5862, -5651, -5634, -5631, 

In [7]:
incident_type = df_paper["incident_type"]
incident_type = incident_type.head(10)
incident_type


0     4
1    13
2    14
3     2
4    14
5    11
6    13
7     2
8    99
9     9
Name: incident_type, dtype: int64

# We keep Events/Seconds/AC-DC/Speed happening 10min after incident and up to 4 hours before an incident. Then we create the new columns in the DataFrame

In [8]:

# Function to convert strings of lists to actual lists
def convert_to_list(column):    return column.apply(lambda x: ast.literal_eval(x))

# Convert columns of the dataframe to actual lists
df_paper['events_sequence'] = convert_to_list(df['events_sequence'])
df_paper['seconds_to_incident_sequence'] = convert_to_list(df['seconds_to_incident_sequence'])
df_paper["train_kph_sequence"] = convert_to_list(df['train_kph_sequence'])
df_paper["dj_ac_state_sequence"] =convert_to_list(df['dj_ac_state_sequence'])
df_paper["dj_dc_state_sequence"] =convert_to_list(df['dj_dc_state_sequence'])

# Define the filter function for events based on time range
def filter_events_tuple_events_seconds(events_sequence, seconds_to_incident_sequence, min_time=-14400, max_time=600):
    return [[event,seconds] for event, seconds in zip(events_sequence, seconds_to_incident_sequence) 
            if min_time <= seconds <= max_time]
# Apply the filter_events_tuple_events_seconds function to each row
df_paper['filtered_events_sequence_with_seconds'] = df_paper.apply(
    lambda row: filter_events_tuple_events_seconds(row['events_sequence'], row['seconds_to_incident_sequence']),
    axis=1
)
df_paper['filtered_ac_sequence_with_seconds'] = df_paper.apply(
    lambda row: filter_events_tuple_events_seconds(row['dj_ac_state_sequence'], row['seconds_to_incident_sequence']),
    axis=1
)
df_paper['filtered_dc_sequence_with_seconds'] = df_paper.apply(
    lambda row: filter_events_tuple_events_seconds(row['dj_dc_state_sequence'], row['seconds_to_incident_sequence']),
    axis=1
)
df_paper['filtered_kph_sequence_with_seconds'] = df_paper.apply(
    lambda row: filter_events_tuple_events_seconds(row['train_kph_sequence'], row['seconds_to_incident_sequence']),
    axis=1
)
#Now we only select the feature in [ feature, seoncds] obtained from the precedent step
def filter_events(events_sequence, seconds_to_incident_sequence, min_time=-14400, max_time=600):
    return [event for event, seconds in zip(events_sequence, seconds_to_incident_sequence) 
            if min_time <= seconds <= max_time]
def filter_seconde(seconds_to_incident_sequence, min_time=-14400, max_time=600):
    return [seconde for seconde in seconds_to_incident_sequence 
            if min_time <= seconde <= max_time]
def filter_ac(dj_ac_state_sequence, seconds_to_incident_sequence, min_time=-14400, max_time=600):
    return [ac for ac, seconds in zip(dj_ac_state_sequence, seconds_to_incident_sequence) 
            if min_time <= seconds <= max_time]
def filter_dc(dj_dc_state_sequence, seconds_to_incident_sequence, min_time=-14400, max_time=600):
    return [dc for dc, seconds in zip(dj_dc_state_sequence, seconds_to_incident_sequence) 
            if min_time <= seconds <= max_time]
def filter_kph(train_kph_sequence, seconds_to_incident_sequence, min_time=-14400, max_time=600):
    return [kph for kph, seconds in zip(train_kph_sequence, seconds_to_incident_sequence) 
            if min_time <= seconds <= max_time]


# Apply the filter function to each row
df_paper['filtered_events_sequence'] = df_paper.apply(
    lambda row: filter_events(row['events_sequence'], row['seconds_to_incident_sequence']),
    axis=1
)
# Apply the filter function to each row
df_paper['filtered_seconde_sequence'] = df_paper.apply(
    lambda row: filter_seconde(row['seconds_to_incident_sequence']),
    axis=1
)
# Apply the filter_ function to each row
df_paper['filtered_ac_sequence'] = df_paper.apply(
    lambda row: filter_ac(row['dj_ac_state_sequence'], row['seconds_to_incident_sequence']),
    axis=1
)
df_paper['filtered_dc_sequence'] = df_paper.apply(
    lambda row: filter_dc(row['dj_dc_state_sequence'], row['seconds_to_incident_sequence']),
    axis=1
)
df_paper['filtered_kph_sequence'] = df_paper.apply(
    lambda row: filter_kph(row['train_kph_sequence'], row['seconds_to_incident_sequence']),
    axis=1
)




## SCATTER PLOTS OF EVENTS BETWEEN -14400 AND -600, ! NOT FILTERED BY METRIC r !

### Speed-Seconds Scatter Plot Around the Incident: Lines Connect Events Occurring on the Same DataFrame Line

In [52]:

def plot_relation_seconds_times() : 
    """
    This function aggregates events, seconds, AC and DC states, and the DataFrame row index (j) 
    into a dictionary to facilitate graphical representation and descriptive statistical analysis.
    """
    count = 0
    vitesse_moment_incident_dico = {}
    dico_event_100kph = {}
    # Iterate over each unique incident type in the DataFrame
    for incident in df_paper["incident_type"].unique():
        vitesse_moment_incident_dico[incident] = []
        dico_event_100kph[incident] =[]
        # Iterate over rows of the DataFrame while keeping track of the row index (j)
        # This ensures that events from the same row are grouped for line continuity in the plot
        for j, row in df_paper.iterrows():
            if row["incident_type"] == incident:
                secondes_autour_incident = row["filtered_seconde_sequence"]
                # Initialize lists to store data within the time window
                liste_des_seoncde_atteintes = []
                vitesse_liste = []
                ac_liste=[]
                dc_liste=[]
                events_liste = []
                # Filter data based on the time window (-600 to 600 seconds)
                for i, sec in enumerate(secondes_autour_incident):
                    if -600 <= int(sec) <= 600:  # Fenêtre de temps autour de l'incident
                        count += 1
                        vitesse_moment_incident = row["filtered_kph_sequence"][i]
                        ac_moment_incident =row["filtered_ac_sequence"][i]
                        dc_moment_incident = row["filtered_dc_sequence"][i]
                        liste_des_seoncde_atteintes.append(sec)
                        vitesse_liste.append(vitesse_moment_incident)
                        ac_liste.append(ac_moment_incident)
                        dc_liste.append(dc_moment_incident)
                        event = row["filtered_events_sequence"][i]
                        events_liste.append(event)
                
                if liste_des_seoncde_atteintes:
                    for sec, vitesse , ac , dc , event  in zip(liste_des_seoncde_atteintes, vitesse_liste, ac_liste, dc_liste, events_liste) : 
                        # Add the tuple to the dictionary for this incident type
                        vitesse_moment_incident_dico[incident].append((( sec, vitesse,ac, dc ,  j, event )))
    
    return vitesse_moment_incident_dico
# Call the function and extract the first returned dictionary
vitesse_moment_incident_dico = plot_relation_seconds_times()

print({key: value[:10] for key, value in vitesse_moment_incident_dico.items()})

{np.int64(4): [(-586, 0.0, False, True, 0, 2708), (-566, 0.0, False, True, 0, 3082), (-566, 0.0, False, True, 0, 4394), (-561, 0.0, False, True, 0, 3086), (-561, 0.0, False, True, 0, 1286), (-555, 0.0, False, True, 0, 1720), (-555, 0.0, False, True, 0, 1740), (-555, 0.0, False, True, 0, 1760), (-555, 0.0, False, True, 0, 1780), (-551, 0.0, False, True, 0, 4396)], np.int64(13): [(-597, 46.6, False, True, 1, 2956), (-567, 27.3, False, True, 1, 2956), (-563, 27.1, False, True, 1, 2956), (-546, 0.0, False, True, 1, 4066), (-529, 0.0, False, True, 1, 3636), (-465, 28.7, False, True, 1, 2956), (-452, 48.2, False, True, 1, 2956), (-439, 63.4, False, True, 1, 2956), (-419, 79.8, False, True, 1, 2956), (-417, 81.1, False, True, 1, 2956)], np.int64(14): [(292, 0.0, False, True, 2, 4028), (292, 0.0, False, True, 2, 4050), (293, 0.0, False, True, 2, 2752), (293, 0.0, False, True, 2, 4026), (293, 0.0, False, True, 2, 4032), (472, 0.0, False, True, 2, 2742), (473, 0.0, False, True, 2, 2708), (477, 0

### Plot: Speed-Time Relationships : legend only for events !=2956 and events occuring only if the train speed >0. Though all events are still in the window

In [ ]:



# Get the dictionary of incident speeds and times using the plot_relation_seconds_times function
vitesse_moment_incident_dico = plot_relation_seconds_times()
# Display the result
print(vitesse_moment_incident_dico)

# List to store the positions of events already annotated
annotations_position = []

# Function to offset annotations slightly (alternating up and down)
def get_offset(idx):
    """This function adjusts the display of events, alternating between placing them above or below"""
    if idx % 2 == 0:
        return -10, 0  # Offset left by 10 pixels
    else:
        return 10, 0   # Offset right by 10 pixels

# Rare incident types to focus on
rares = [17, 16, 7, 6, 11]

"""Start plotting the graph"""
for incident_key in vitesse_moment_incident_dico: 
    if incident_key in rares:   # Focus only on rare incidents to avoid overloaded graphs
        # Extract data for the current incident
        data = vitesse_moment_incident_dico.get(incident_key, [])
        
        # Initialize lists to store times, speeds, AC states, DC states, and events for plotting
        temps_j = []
        vitesses_j = []
        ac_j = []
        dc_j = []
        events_j = []

        # Variable to check if the index 'line' has changed
        previous_index = None

        # Create a new plot for each incident
        plt.figure(figsize=(14, 6))  

        # Extract times, speeds, and events from the tuples
        for entry in data:
            time, vitesse, ac, dc, line, event = entry  # Decompose each tuple into time, speed, AC, DC, index, and event

            # If the index has changed, plot the previous points and reset the lists
            if previous_index is not None and line != previous_index:
                if temps_j and vitesses_j:
                    # Plot the points for the previous index
                    line_plot = plt.plot(temps_j, vitesses_j, marker='o', linestyle='-', label=f'Incident {incident_key} - Index {previous_index}')
                    line_color = line_plot[0].get_color()  # Get the color of the line for annotations
                    
                    # Annotate the points with events (skip event 2956 or speed 0)
                    offset_idx = 0  # Index to apply alternate offset
                    for idx, (tx, ty, ev, v) in enumerate(zip(temps_j, vitesses_j, events_j, vitesses_j)):
                        if ev != 2956 and v != 0:  # Skip event 2956 and speed 0
                            # Apply alternating offset for non-excluded events
                            offset_x, offset_y = get_offset(offset_idx)

                            # Annotate the event with the calculated offset
                            plt.annotate(str(ev), (tx, ty), textcoords="offset points", xytext=(offset_x, offset_y), ha='center', color=line_color)
                            annotations_position.append((tx + offset_x, ty + offset_y))  # Store the new annotation position

                            # Increment the offset index
                            offset_idx += 1

                # Reset lists for the new index
                temps_j = []
                vitesses_j = []
                ac_j = []
                dc_j = []
                events_j = []

            # Add points for the current index
            temps_j.append(time)
            vitesses_j.append(vitesse)
            ac_j.append(ac)
            dc_j.append(dc)
            events_j.append(event)  # Add the event corresponding to the current index
            previous_index = line  # Update the previous index

        # After the loop, plot the last set of points for the final index
        if temps_j and vitesses_j:
            line_plot = plt.plot(temps_j, vitesses_j, marker='o', linestyle='-', label=f'Incident {incident_key} - Index {previous_index}')
            line_color = line_plot[0].get_color()  # Get the color of the line for annotations

            # Annotate the points with events (skip event 2956 or speed 0)
            offset_idx = 0  # Reset offset index
            for idx, (tx, ty, ev, v) in enumerate(zip(temps_j, vitesses_j, events_j, vitesses_j)):
                if ev != 2956 and v != 0:  # Skip event 2956 and speed 0
                    # Apply alternating offset for non-excluded events
                    offset_x, offset_y = get_offset(offset_idx)

                    # Annotate the event with the calculated offset
                    plt.annotate(str(ev), (tx, ty), textcoords="offset points", xytext=(offset_x, offset_y), ha='center', color=line_color)
                    annotations_position.append((tx + offset_x, ty + offset_y))  # Store the new annotation position

                    # Increment the offset index
                    offset_idx += 1

        # Add labels and a title to the graph
        plt.title(f"Speed over Time for Incident {incident_key}")
        plt.xlabel("Time (seconds)")
        plt.ylabel("Speed (km/h)")

        # Set X-axis limit to range from -1100 to 1100
        plt.xlim(-1100, 1100)

        plt.legend()

        # Add grid to improve readability
        plt.grid(True)


        #plt.show()


{np.int64(4): [(-586, 0.0, False, True, 0, 2708), (-566, 0.0, False, True, 0, 3082), (-566, 0.0, False, True, 0, 4394), (-561, 0.0, False, True, 0, 3086), (-561, 0.0, False, True, 0, 1286), (-555, 0.0, False, True, 0, 1720), (-555, 0.0, False, True, 0, 1740), (-555, 0.0, False, True, 0, 1760), (-555, 0.0, False, True, 0, 1780), (-551, 0.0, False, True, 0, 4396), (-547, 0.0, False, True, 0, 1286), (-546, 0.0, False, True, 0, 2652), (-545, 0.0, False, True, 0, 4094), (-473, 0.0, False, True, 0, 2742), (-473, 0.0, False, True, 0, 4026), (-473, 0.0, False, True, 0, 4148), (-451, 0.0, False, True, 0, 2708), (-443, 0.0, False, True, 0, 3036), (-443, 0.0, False, True, 0, 4394), (-413, 0.0, False, True, 0, 4168), (-411, 0.0, False, True, 0, 4140), (-395, 0.0, False, False, 0, 3986), (219, 0.0, False, False, 0, 2744), (219, 0.0, False, False, 0, 4002), (222, 0.0, False, False, 0, 2852), (222, 0.0, False, False, 0, 4110), (223, 0.0, False, False, 0, 2854), (224, 0.0, False, False, 0, 2982), (224

### Speed-time relationship but here we don't display the events so it may be more readable for classes occuring many times 

In [ ]:
# Initialize the figure for the histogram
for incident_key, data in vitesse_moment_incident_dico.items():
    # Create a dictionary to group data by the index "line"
    line_groups = {}
    
    # Group data by "line"
    for entry in data:
        time, vitesse, ac, dc, line, event = entry  # Decompose each tuple into time, speed, AC, DC, line, and event
        if line not in line_groups:
            line_groups[line] = {'temps': [], 'vitesses': []}  # Initialize the groups for each line
        line_groups[line]['temps'].append(time)  # Add time data for this line
        line_groups[line]['vitesses'].append(vitesse)  # Add speed data for this line

    # Create a new plot for the current incident
    plt.figure(figsize=(10, 6))  # Set the figure size for better visibility

    # Plot the lines for each group (line)
    for line, group in line_groups.items():
        temps = group['temps']  # Extract time data for this line
        vitesses = group['vitesses']  # Extract speed data for this line

        # Plot the points and connect the points for the same line
        plt.plot(temps, vitesses, marker='o', label=f'Line {line}', alpha=0.7)  # Use a line with markers and set transparency

    # Add labels and title to the graph
    plt.title(f"Speed over Time for Incident {incident_key}")  # Title based on the incident key
    plt.xlabel("Time (seconds)")  # Label for the x-axis
    plt.ylabel("Speed (km/h)")  # Label for the y-axis


    plt.legend()

    plt.grid(True)


    #plt.show()



## Speed-time relationship without lines joining the events. Here we kept only events occuring when the train was at speed >= 80kph. Events in green are 2956. Take a look at the incident 11 : we can observe a particular sequence of events that looks like  the graph of f(x)  = x (in the negative domain)

In [ ]:
dico_event_100kph = {}

# Iterate through each incident type
for incident in df_paper["incident_type"].unique():
    # Initialize dictionaries for this incident
    dico_event_100kph[incident] = []
    df_paper["filtered_seconde_sequence"]  # Data for filtered second sequence is not being used here

    # Iterate through each row in the DataFrame
    for j, row in df_paper.iterrows():
        evventseq = row["filtered_events_sequence"]
        vitesseseq = row["filtered_kph_sequence"]
        secondseq = row["filtered_seconde_sequence"]
        
        if row["incident_type"] == incident:
            # Iterate through the events, speeds, and times associated with each row
            for event, vitesse, seconde in zip(evventseq, vitesseseq, secondseq): 
                # Filter events with speed >= 80 and time between -600 and 600 seconds
                if vitesse >= 80 and -600 < seconde < 600:
                    dico_event_100kph[incident].append((event, vitesse, seconde, j))

# Import necessary libraries for plotting



# Choose the incidents for which you want to plot the graphs (it may take a while for recurent incidents)
incident_keys = [4, 16, 6, 17, 7]

# Loop through each incident
for incident_key in dico_event_100kph:
    # Extract data for this incident
    data = dico_event_100kph.get(incident_key, [])

    # Initialize lists to store data
    temps_j = []  # Time (X)
    vitesses_j = []  # Speed (Y)
    events_j = []  # Events
    colors = []  # Colors associated with events
    sizes = []  # Size of points to make certain events more "flashy"

    # Define a color palette (one color per event, except for event 2956)
    unique_events = list(set(event for event, _, _, _ in data))  # Unique events
    event_colors = {event: plt.cm.tab20(i / len(unique_events)) for i, event in enumerate(unique_events)}
    event_colors[2956] = "green"  # Set color of event 2956 to green

    # Count occurrences of each event
    event_counts = Counter(event for event, _, _, _ in data)

    # Sort events by descending frequency
    sorted_events = sorted(event_counts.items(), key=lambda x: x[1], reverse=True)

    # Extract data for each point
    for event, vitesse, seconde, _ in data:
        temps_j.append(seconde)  # Time
        vitesses_j.append(vitesse)  # Speed
        events_j.append(event)  # Event

        # Make events different from 2956 more "flashy"
        if event != 2956:
            colors.append(event_colors[event])  # Unique color
            sizes.append(120)  # Larger size for these points
        else:
            colors.append("green")  # Green for event 2956
            sizes.append(30)  # Normal size for event 2956

    # Create the plot
    fig, ax = plt.subplots(figsize=(20, 10))

    # Plot the points on the main graph
    scatter = ax.scatter(temps_j, vitesses_j, c=colors, s=sizes, label='Points', alpha=0.9, edgecolor='k')

    # Add annotations for events other than 2956
    for i, event in enumerate(events_j):
        if event != 2956:
            ax.annotate(
                str(event),  # Text to display (event name)
                (temps_j[i], vitesses_j[i]),  # Position of the point
                textcoords="offset points",  # Offset text position
                xytext=(5, 5),  # Pixel offset (x, y)
                fontsize=10,  # Font size
                color=event_colors[event]  # Text color
            )

    # Add event annotations in the margin, sorted by descending frequency
    sorted_patches = [
        mpatches.Patch(color=event_colors[event], label=f"Event {event} ({count})")
        for event, count in sorted_events
    ]
    plt.legend(handles=sorted_patches, title="Events (sorted by descending frequency)", loc='upper left', bbox_to_anchor=(1.05, 1), fontsize=10)


    ax.set_title(f"Speed vs Time for Incident {incident_key}", fontsize=16)
    ax.set_xlabel("Time (seconds)", fontsize=12)
    ax.set_ylabel("Speed (km/h)", fontsize=12)

    ax.grid(True)

    plt.tight_layout()
    #plt.show()


## Time-Speed-AC-DC Plot: lines joining AC-DC state represent AC-DC state on a same line in the DataFrame (as above)

In [ ]:


# List of specific incident keys to process
m = [6, 4, 14 , 3, 16, 17, 7]
present_ac = False
present_dc = False
present_ac_dc = False
absent = False

# Loop through each incident key and its corresponding data
for incident_key, data in vitesse_moment_incident_dico.items():
    if incident_key in m :
        # Dictionary to group the data by "line"
        line_groups = {}

        # Group the data by "line" (index of the line)
        for entry in data:
            time, vitesse, ac, dc, line, event = entry  # Decompose each tuple
            if line not in line_groups:
                line_groups[line] = {'temps': [], 'vitesses': [], 'ac_dc': []}  # Also store AC and DC
            line_groups[line]['temps'].append(time)
            line_groups[line]['vitesses'].append(vitesse)
            line_groups[line]['ac_dc'].append((ac, dc))  # Add the AC and DC pair

        # Initialize legend elements for each AC-DC combination
        legend_elements = {}

        # Plot the data for each line
        for line, group in line_groups.items():
            temps = group['temps']
            vitesses = group['vitesses']
            ac_dc = group['ac_dc']

            # Sort the points by time (chronological order)
            sorted_indices = sorted(range(len(temps)), key=lambda i: temps[i])
            sorted_temps = [temps[i] for i in sorted_indices]
            sorted_vitesses = [vitesses[i] for i in sorted_indices]
            sorted_ac_dc = [ac_dc[i] for i in sorted_indices]

            # Connect points with a line while applying colors based on AC and DC
            for i in range(len(sorted_temps)):
                ac, dc = sorted_ac_dc[i]

                # Determine the color based on the AC and DC conditions
                if ac and dc:  # AC=True and DC=True
                    color = '#00FFFF'  # Cyan color
                    present_ac_dc = True
                    legend_elements['AC=True, DC=True'] = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='#00FFFF', label='AC=True, DC=True')
                elif (not dc) and ac:  # AC=True and DC=False
                    color = 'darkblue'
                    present_ac = True
                    legend_elements['AC=True, DC=False'] = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='darkblue', label='AC=True, DC=False')
                elif (not ac) and dc:  # AC=False and DC=True
                    color = '#7CFC00'  # Light green
                    present_dc = True
                    legend_elements['AC=False, DC=True'] = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='#7CFC00', label='AC=False, DC=True')
                else:  # AC=False and DC=False
                    color = 'red'
                    absent = True
                    legend_elements['AC=False, DC=False'] = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', label='AC=False, DC=False')

                # Plot each point with the appropriate color only if the condition is "True"
                if (present_ac_dc and ac and dc) or (present_ac and ac and not dc) or (present_dc and not ac and dc) or (absent and not ac and not dc):
                    plt.plot(sorted_temps[i], sorted_vitesses[i], marker='o', color=color, alpha=0.7)

            # Connect the points of the same line with a black line
            plt.plot(sorted_temps, sorted_vitesses, color='black', alpha=0.5)  # Black line to connect the points

        plt.title(f"Speed vs Time for Incident {incident_key}")
        plt.xlabel("Time (seconds)")
        plt.ylabel("Speed (km/h)")

        if present_ac_dc or present_ac or present_dc or absent:
            plt.legend(handles=list(legend_elements.values()))

        plt.grid(True)
        #plt.show()

        # Reset boolean variables for the next plot
        present_ac = present_dc = present_ac_dc = absent = False



c:\Users\matin\anaconda3\envs\sfml\Lib\tkinter\__init__.py:862: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)


# Data mining Part

### Percentage of the occurence of classes in the DataFrame 

In [14]:
incident_type_percentage = df_paper["incident_type"].value_counts(normalize=True) * 100
incident_type_percentage_df = incident_type_percentage.reset_index()
incident_type_percentage_df.columns = ['Incident Type', 'Percentage']

incident_type_percentage_df

,Incident Type,Percentage
0,13,31.454006
1,99,17.309594
2,14,14.737883
3,2,11.770524
4,9,11.572700
5,4,7.715134
6,11,2.571711
7,17,0.989120
8,6,0.593472
9,3,0.494560


In [15]:
incident_type_percentage = df_paper["incident_type"].value_counts(normalize=True) * 100
plt.figure(figsize=(8, 6))
incident_type_percentage.plot(kind='bar')
plt.title("Percentage of occurrence of each type of incident")
plt.xlabel("Incident Type")
plt.ylabel("Percentage ")
plt.xticks(rotation=45)
plt.show()

# Model 
## F1-score around 90%

## Filtering event (not all of them are useful for incident diagnostics)

In [16]:
from collections import Counter
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# Step 1: Calculate event frequencies in each class and across all classes
def calculate_event_frequencies(df):
    class_event_freq = {}
    overall_event_freq = Counter()
    
    for incident_class in df['incident_type'].unique():
        class_data = df[df['incident_type'] == incident_class]
        event_counter = Counter([event[0] for seq in class_data['filtered_events_sequence_with_seconds'] for event in seq]) 
        class_event_freq[int(incident_class)] = event_counter
        overall_event_freq.update(event_counter)
    
    return class_event_freq, overall_event_freq

class_event_freq, overall_event_freq = calculate_event_frequencies(df_paper)

In [51]:
#class_event_freq
from collections import Counter

# Créer un nouveau Counter avec seulement les 10 premiers
{key: Counter(dict(class_event_freq[key].most_common(10))) for key in class_event_freq}

{4: Counter({2956: 17299,
          3658: 1542,
          3636: 1530,
          4068: 1504,
          4066: 1433,
          4026: 1026,
          2708: 640,
          4120: 548,
          4124: 504,
          4020: 473}),
 13: Counter({2956: 77866,
          3658: 6988,
          3636: 6986,
          4068: 5948,
          4066: 5858,
          4026: 2234,
          4124: 1951,
          4120: 1794,
          2708: 1561,
          4168: 1078}),
 14: Counter({2956: 32230,
          3658: 2552,
          3636: 2548,
          4066: 2086,
          4068: 2037,
          4168: 1600,
          4026: 1265,
          4140: 1147,
          2708: 840,
          4120: 693}),
 2: Counter({2956: 42004,
          3658: 4080,
          3636: 4008,
          4068: 3233,
          4066: 3209,
          4026: 1761,
          2708: 1217,
          4120: 1108,
          4124: 989,
          4020: 759}),
 11: Counter({2956: 6890,
          3658: 509,
          3636: 498,
          4066: 473,
          406

In [50]:
#overall_event_freq
Counter(dict(overall_event_freq.most_common(10)))

Counter({2956: 244955,
         3658: 22351,
         3636: 22232,
         4068: 19128,
         4066: 18994,
         4026: 9205,
         2708: 6068,
         4124: 5918,
         4120: 5824,
         4168: 4358})

In [49]:
# Step 2: Calculate relevance metric r for each event per class
def calculate_relevance_metric(class_event_freq, overall_event_freq):
    
    relevance_metric = {}
    
    for incident_class, event_counts in class_event_freq.items():
        relevance_metric[incident_class] = {}
        for event, freq_in_class in event_counts.items():
            freq_in_all_classes = overall_event_freq[event]
            relevance_metric[int(incident_class)][event] = freq_in_class / freq_in_all_classes 
    
    return relevance_metric

relevance_metric = calculate_relevance_metric(class_event_freq, overall_event_freq)


{key: dict(list(value.items())[:10]) for key, value in relevance_metric.items()}

{4: {2744: 0.09239130434782608,
  4004: 0.13408723747980614,
  2852: 0.12643106022896963,
  4110: 0.13643926788685523,
  2854: 0.1259920634920635,
  4396: 0.10979794128860083,
  1132: 0.2708333333333333,
  4140: 0.1095183486238532,
  4148: 0.10845706907682376,
  2708: 0.1054713249835201},
 13: {2744: 0.27801003344481606,
  4148: 0.2830858618463525,
  4394: 0.25452856479331165,
  1566: 0.199203187250996,
  1570: 0.199203187250996,
  4396: 0.3564620663362562,
  3634: 0.1906474820143885,
  4124: 0.32967218654950997,
  2956: 0.3178787940642159,
  4068: 0.3109577582601422},
 14: {4394: 0.14677194612169067,
  1566: 0.14940239043824702,
  1570: 0.14940239043824702,
  4114: 0.2684931506849315,
  4168: 0.36714089031665903,
  4156: 0.2622568093385214,
  4066: 0.10982415499631462,
  4068: 0.1064930991217064,
  2540: 0.425,
  4140: 0.32884174311926606},
 2: {4066: 0.16894808887016952,
  4068: 0.16901923881221245,
  2742: 0.17524457677584007,
  4026: 0.1913090711569799,
  2708: 0.20056031641397495,

### We now concentrate our attention on EVENTS-SECONDS-AC-DC-SPEED for events which have relevance metric r >= 0.2. (This treshold will be determined after actually)

In [48]:
# Step 3: Filter events based on threshold tr to maximize F1-score
def filter_events(relevance_metric, threshold):
    # Initialize an empty dictionary to store filtered events
    filtered_events = {}
    
    # Iterate over each incident class and their event metrics
    for incident_class, event_metrics in relevance_metric.items():
        # For each incident class, filter events whose relevance metric is above the threshold
        filtered_events[incident_class] = [event for event, r in event_metrics.items() if r >= threshold]
        
    return filtered_events


# Set an initial threshold tr
threshold_tr = 0.2
# Filter events based on relevance metric and threshold
filtered_events = filter_events(calculate_relevance_metric(calculate_event_frequencies(df_paper)[0], calculate_event_frequencies(df_paper)[1]), 0.2)


#print(filtered_events)

# Dictionary to store filtered events by incident type
events_dict_by_type = {}

# Function to filter events in a row based on the threshold and event type
def filter_events_row_train(row, filtered_events):
    # Retrieve the events sequence and incident type from the row
    events_sequence = row['filtered_events_sequence']
    incident_type = row['incident_type']
    
    # Check if the incident type is in the filtered events dictionary
    if incident_type in filtered_events:
        # Keep only the events that are in filtered_events for the given incident type
        return [event for event in events_sequence if event in filtered_events[incident_type]]
    else:
        # Return an empty list if the incident type is not found in filtered_events
        return []

# Apply filtering to the 'filtered_events_sequence' column in the DataFrame
df_paper['filtered_events_metric_r'] = df_paper.apply(
    lambda row: filter_events_row_train(row, filtered_events), axis=1
)

# Function to filter events and seconds for each row based on event relevance
def filter_events_seconds_row_train(row, filtered_events):
    """Filter events and seconds based on their relevance metric"""
    # Retrieve the event sequence and second sequence from the row
    events_sequence = row['filtered_events_sequence']
    seconds_sequence = row['filtered_seconde_sequence']
    incident_type = row['incident_type']
  
    # Check if the incident type is in the filtered events dictionary
    if incident_type in filtered_events:
        # Filter events and their associated seconds
        filtered_events_with_seconds = [
            (event, sec) for event, sec in zip(events_sequence, seconds_sequence)
            if event in filtered_events[incident_type]
        ]
        # Separate the filtered events and seconds into two lists
        filtered_events_filtered = [event for event, sec in filtered_events_with_seconds]
        filtered_seconds_filtered = [sec for event, sec in filtered_events_with_seconds]
        
        return filtered_events_filtered, filtered_seconds_filtered
    else:
        # Return empty lists if the incident type is not found in filtered_events
        return [], []

# Function to filter 'ac' values based on the event relevance
def filter_ac_row_train(row, filtered_events):
    """Filter 'ac' values based on the event relevance"""
    # Retrieve the event sequence and incident type from the row
    events_sequence = row['filtered_events_sequence']
    incident_type = row['incident_type']
    ac_sequence = row["filtered_ac_sequence"]
    
    # Check if the incident type is in the filtered events dictionary
    if incident_type in filtered_events:
        # Keep only the 'ac' values corresponding to the filtered events
        return [ac for event, ac in zip(events_sequence, ac_sequence) if event in filtered_events[incident_type]]
    else:
        # Return an empty list if the incident type is not found in filtered_events
        return []

# Function to filter 'dc' values based on the event relevance
def filter_dc_row_train(row, filtered_events):
    """Filter 'dc' values based on the event relevance"""
    # Retrieve the event sequence and incident type from the row
    events_sequence = row['filtered_events_sequence']
    incident_type = row['incident_type']
    dc_sequence = row["filtered_dc_sequence"]
    
    # Check if the incident type is in the filtered events dictionary
    if incident_type in filtered_events:
        # Keep only the 'dc' values corresponding to the filtered events
        return [dc for event, dc in zip(events_sequence, dc_sequence) if event in filtered_events[incident_type]]
    else:
        # Return an empty list if the incident type is not found in filtered_events
        return []

# Function to filter 'kph' values based on the event relevance
def filter_kph_row_train(row, filtered_events):
    """Filter 'kph' values based on the event relevance"""
    # Retrieve the event sequence and incident type from the row
    events_sequence = row['filtered_events_sequence']
    incident_type = row['incident_type']
    kph_sequence = row["filtered_kph_sequence"]
    
    # Check if the incident type is in the filtered events dictionary
    if incident_type in filtered_events:
        # Keep only the 'kph' values corresponding to the filtered events
        return [kph for event, kph in zip(events_sequence, kph_sequence) if event in filtered_events[incident_type]]
    else:
        # Return an empty list if the incident type is not found in filtered_events
        return []

# Function to filter seconds based on the event relevance
def filter_second_row_train(row, filtered_events):
    """Filter seconds based on the event relevance"""
    # Retrieve the event sequence and seconds sequence from the row
    events_sequence_ = row['filtered_events_sequence']
    seconde_ = row["filtered_seconde_sequence"]
    incident_type = row['incident_type']
   
    # Check if the incident type is in the filtered events dictionary
    if incident_type in filtered_events:
        # Keep only the seconds corresponding to the filtered events
        return [sec for event, sec in zip(events_sequence_, seconde_) if event in filtered_events[incident_type]]
    else:
        # Return an empty list if the incident type is not found in filtered_events
        return []

# Apply the filtering to each relevant column in the DataFrame
df_paper['filtered_ac_metric_r'] = df_paper.apply(
    lambda row: filter_ac_row_train(row, filtered_events), axis=1)
df_paper['filtered_dc_metric_r'] = df_paper.apply(
    lambda row: filter_dc_row_train(row, filtered_events), axis=1)
df_paper['filtered_kph_metric_r'] = df_paper.apply(
    lambda row: filter_kph_row_train(row, filtered_events), axis=1)
df_paper['filtered_seconds_metric_r'] = df_paper.apply(
    lambda row: filter_second_row_train(row, filtered_events), axis=1)



# Plots for events ! filtered by metric r !

### Plot ! after applying the metric r ! : Speed-Time Relationships : legend only for events !=2956 and events occuring only if the train's speed >0. Notice that for certain classes there are very little or no events after the filtering with metric r though classes tend to conserve their respective pattern e.g, class 6, 11, 14,...

In [ ]:

def plot_relation_seconds_times_metric_r() : 
    """Same as above, but with the metric r"""
    count = 0
    plot_relation_seconds_times_metric_r = {}
    dico_event_100kph_metric_r = {}
    # Loop through each unique incident type
    for incident in df_paper["incident_type"].unique():
        plot_relation_seconds_times_metric_r[incident] = []
        dico_event_100kph_metric_r[incident] = []
        # Loop through the rows of the DataFrame
        for j, row in df_paper.iterrows():
            if row["incident_type"] == incident:
                
                # List of seconds around the incident
                secondes_autour_incident = row["filtered_seconds_metric_r"]
                
                # Initialize lists to store the negative and positive seconds
                liste_des_seoncde_atteintes = []
                vitesse_liste = []
                ac_liste = []
                dc_liste = []
                events_liste = []

                # Check the speed near the incident
                for i, sec in enumerate(secondes_autour_incident):
                    if -250 <= int(sec) <= 250:  # Time window around the incident
                       
                        count += 1
                        vitesse_moment_incident = row["filtered_kph_metric_r"][i]
                        ac_moment_incident = row["filtered_ac_metric_r"][i]
                        dc_moment_incident = row["filtered_dc_metric_r"][i]
                     
                        liste_des_seoncde_atteintes.append(sec)
                        vitesse_liste.append(vitesse_moment_incident)
                        ac_liste.append(ac_moment_incident)
                        dc_liste.append(dc_moment_incident)
                        event = row["filtered_events_metric_r"][i]
                        events_liste.append(event)
                
                if liste_des_seoncde_atteintes:
                    for sec, vitesse , ac , dc , event in zip(liste_des_seoncde_atteintes, vitesse_liste, ac_liste, dc_liste, events_liste): 
                        # Add the tuple (supremum, speed_supremum, infimum, speed_infimum) to the dictionary
                        plot_relation_seconds_times_metric_r[incident].append((( sec, vitesse, ac, dc ,  j, event )))
    
    return plot_relation_seconds_times_metric_r
vitesse_moment_incident_dicom_metric_r = plot_relation_seconds_times_metric_r()
# Display the results
print(plot_relation_seconds_times_metric_r)

# List of already placed annotations (event positions)
annotations_position = []

# Function to slightly offset annotations
def get_offset(idx):
    if idx % 2 == 0:
        return -10, 0  # Offset to the left by 10 pixels
    else:
        return 10, 0   # Offset to the right by 10 pixels


"""Some incident have almost no events after the filtering e.g, 16, others still share the same patterns as above e.g, 11 or 13 
Choose here your incidents of interest or just replace the list by 'vitesse_moment_incident_dicom_metric_r.items() ' """
for incident_key in [4, 13, 14, 16, 6, 17, 7, 11, 6]: 
    # Extract data for this incident
    data = vitesse_moment_incident_dicom_metric_r.get(incident_key, [])

    # Initialize lists to store times, speeds, and points to plot
    temps_j = []
    vitesses_j = []
    ac_j = []
    dc_j = []
    events_j = []

    # Flag to check if the index j has changed
    previous_index = None

 
    plt.figure(figsize=(14, 6))  # Increase width (14) and adjust height (6)

    # Extract times, speeds, and events from the tuples
    for entry in data:
        time, vitesse, ac, dc, line, event = entry  # Decompose each tuple into time, speed, AC, DC, index, and event

        # If the index changes, plot the previous points and reset the lists
        if previous_index is not None and line != previous_index:
            if temps_j and vitesses_j:
                # Plot points for the previous index
                line_plot = plt.plot(temps_j, vitesses_j, marker='o', linestyle='-', label=f'Incident {incident_key} - Index {previous_index}')
                line_color = line_plot[0].get_color()  # Get the color of the plotted line
                
                # Annotate the points with events (do not annotate if event is 2956 or speed is 0)
                offset_idx = 0  # Use an index to determine the offset
                for idx, (tx, ty, ev, v) in enumerate(zip(temps_j, vitesses_j, events_j, vitesses_j)):
                    if ev != 2956 and v != 0:  # Exclude event 2956 and speed 0
                        # Apply an alternate offset depending on the event that is not excluded
                        offset_x, offset_y = get_offset(offset_idx)

                        # Annotate the event with the calculated offset
                        plt.annotate(str(ev), (tx, ty), textcoords="offset points", xytext=(offset_x, offset_y), ha='center', color=line_color)
                        annotations_position.append((tx + offset_x, ty + offset_y))  # Add the new annotation position

                        # Increase the offset index
                        offset_idx += 1

            # Reset for the new index
            temps_j = []
            vitesses_j = []
            ac_j = []
            dc_j = []
            events_j = []

        # Add points for the current index
        temps_j.append(time)
        vitesses_j.append(vitesse)
        ac_j.append(ac)
        dc_j.append(dc)
        events_j.append(event)  # Add the corresponding event
        previous_index = line  # Update the previous index

    # After the loop, there are still points to plot for the last index
    if temps_j and vitesses_j:
        line_plot = plt.plot(temps_j, vitesses_j, marker='o', linestyle='-', label=f'Incident {incident_key} - Index {previous_index}')
        line_color = line_plot[0].get_color()  # Get the color of the plotted line

        # Annotate the points with events (do not annotate if event is 2956 or speed is 0)
        offset_idx = 0  # Reset the offset index
        for idx, (tx, ty, ev, v) in enumerate(zip(temps_j, vitesses_j, events_j, vitesses_j)):
            if ev != 2956 and v != 0:  # Exclude event 2956 and speed 0
                # Apply an alternate offset depending on the event that is not excluded
                offset_x, offset_y = get_offset(offset_idx)

                # Annotate the event with the calculated offset
                plt.annotate(str(ev), (tx, ty), textcoords="offset points", xytext=(offset_x, offset_y), ha='center', color=line_color)
                annotations_position.append((tx + offset_x, ty + offset_y))  # Add the new annotation position

                # Increase the offset index
                offset_idx += 1


    plt.title(f"Speed vs Time for Incident {incident_key}")
    plt.xlabel("Time (seconds)")
    plt.ylabel("Speed (km/h)")

    # Limit the X-axis range to -1100 to 1100
    plt.xlim(-300, 500)


    plt.legend()

    plt.grid(True)

 
    #plt.show()


<function plot_relation_seconds_times_metric_r at 0x0000012C58B06E80>


C:\Users\matin\AppData\Local\Temp\ipykernel_26980\1464014091.py:148: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


## Time-Speed-AC-DC Graphs ! metric r ! : lines joining AC-DC state represent AC-DC state on a same line in the DataFrame (as above)

In [ ]:

incident_of_interest = [9,14,13,9,7,99, 6]
present_ac = False
present_dc = False
present_ac_dc = False
absent = False

# Loop through incidents in the plot_relation_seconds_times_metric_r dictionary
for incident_key, data in vitesse_moment_incident_dicom_metric_r.items() : 
    if incident_key in incident_of_interest:
        # Dictionary to group data by "line"
        line_groups = {}

        # Group data by "line"
        for entry in data:
            time, vitesse, ac, dc, line, event = entry  # Decompose each tuple
            if line not in line_groups:
                line_groups[line] = {'temps': [], 'vitesses': [], 'ac_dc': []}  # Store AC and DC
            line_groups[line]['temps'].append(time)
            line_groups[line]['vitesses'].append(vitesse)
            line_groups[line]['ac_dc'].append((ac, dc))  # Add the (AC, DC) pair

        # Initialize legend elements for each (AC, DC) combination
        legend_elements = {}

        # Plot the data
        for line, group in line_groups.items():
            temps = group['temps']
            vitesses = group['vitesses']
            ac_dc = group['ac_dc']

            # Sort the points by time (chronological order)
            sorted_indices = sorted(range(len(temps)), key=lambda i: temps[i])
            sorted_temps = [temps[i] for i in sorted_indices]
            sorted_vitesses = [vitesses[i] for i in sorted_indices]
            sorted_ac_dc = [ac_dc[i] for i in sorted_indices]

            # Plot each point with colors based on AC and DC
            for i in range(len(sorted_temps)):
                ac, dc = sorted_ac_dc[i]

                # Determine the color based on AC and DC conditions
                if ac and dc:  # AC=True and DC=True
                    color = '#00FFFF'
                    present_ac_dc = True
                    legend_elements['AC=True, DC=True'] = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='#00FFFF', label='AC=True, DC=True')
                elif (not dc) and ac:  # AC=True and DC=False
                    color = 'darkblue'
                    present_ac = True
                    legend_elements['AC=True, DC=False'] = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='darkblue', label='AC=True, DC=False')
                elif (not ac) and dc:  # AC=False and DC=True
                    color = '#7CFC00'
                    present_dc = True
                    legend_elements['AC=False, DC=True'] = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='#7CFC00', label='AC=False, DC=True')
                else:  # AC=False and DC=False
                    color = 'red'
                    absent = True
                    legend_elements['AC=False, DC=False'] = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', label='AC=False, DC=False')

                # Plot each point with the appropriate color if the condition is "True"
                if (present_ac_dc and ac and dc) or (present_ac and ac and not dc) or (present_dc and not ac and dc) or (absent and not ac and not dc):
                    plt.plot(sorted_temps[i], sorted_vitesses[i], marker='o', color=color, alpha=0.7)

            # Connect the points of the same line with a black line
            plt.plot(sorted_temps, sorted_vitesses, color='black', alpha=0.5)  # Black line to connect the points

        plt.title(f"Speed vs Time for Incident {incident_key}")
        plt.xlabel("Time (seconds)")
        plt.ylabel("Speed (km/h)")

  
        if present_ac_dc or present_ac or present_dc or absent:
            plt.legend(handles=list(legend_elements.values()))

 
        plt.grid(True)
        plt.show()

        # Reset the boolean variables for the next plot
        present_ac = present_dc = present_ac_dc = absent = False


plt.title(f"Speed vs Time for Incident {incident_key}")
plt.xlabel("Time (seconds)")
plt.ylabel("Speed (km/h)")


plt.legend(['AC=True, DC=False and AC=False, DC=True'], loc='upper left')


plt.grid(True)
#plt.show()

# Reset the boolean variables for the next plot
present_ac = present_dc = present_ac_dc = absent = False

c:\Users\matin\anaconda3\envs\sfml\Lib\tkinter\__init__.py:862: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)


## ! after filtering with metric r !  Speed-time relationship without lines joining the events. Here we kept only events occuring when the train was at speed >= 80kph. Events in green are 2956. Take a look at the incident 11 : we still can observe a particular sequence of events that looks like the graph ofthe funciton  f(x) = x  (negative domain)

In [ ]:
# Initialize an empty dictionary to store events at or above 100 km/h
dico_event_100kph_metric_r = {}

# Iterate over each unique incident type in the DataFrame
for incident in df_paper["incident_type"].unique():
    # Initialize the list for each incident type in the dictionary
    dico_event_100kph_metric_r[incident] = []
    
    # Loop through each row in the DataFrame
    for j, row in df_paper.iterrows():
        # Extract event sequence, speed sequence, and second sequence
        evventseq = row["filtered_events_metric_r"]
        vitesseseq = row["filtered_kph_metric_r"]
        secondseq = row["filtered_seconds_metric_r"]
        
        # Process rows that match the current incident type
        if row["incident_type"] == incident:
            # Loop through each event, speed, and time pair
            for event, vitesse, seconde in zip(evventseq, vitesseseq, secondseq):
                # Store events that meet the condition (speed >= 80 and -50 < second < 50)
                if vitesse >= 80 and -50 < seconde < 50:
                    dico_event_100kph_metric_r[incident].append((event, vitesse, seconde, j))
# Import necessary libraries for plotting
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from collections import Counter

# Loop through each incident in the dictionary of events
for incident_key in dico_event_100kph_metric_r:
    # Extract data for the current incident
    data = dico_event_100kph_metric_r.get(incident_key, [])

    # Initialize lists to store the data
    temps_j = []  # Time (X-axis)
    vitesses_j = []  # Speed (Y-axis)
    events_j = []  # Events
    colors = []  # Colors associated with events
    sizes = []  # Size of points to make certain events more "flashy"

    # Define a color palette for events (one color per event, except for event 2956)
    unique_events = list(set(event for event, _, _, _ in data))  # Unique events
    event_colors = {event: plt.cm.tab20(i / len(unique_events)) for i, event in enumerate(unique_events)}
    event_colors[2956] = "green"  # Fix color for event 2956 to green

    # Count the occurrences of each event
    event_counts = Counter(event for event, _, _, _ in data)

    # Sort the events by frequency in descending order
    sorted_events = sorted(event_counts.items(), key=lambda x: x[1], reverse=True)

    # Extract data for each point
    for event, vitesse, seconde, _ in data:
        temps_j.append(seconde)  # Time
        vitesses_j.append(vitesse)  # Speed
        events_j.append(event)  # Event

        # Make events other than 2956 more "flashy"
        if event != 2956:
            colors.append(event_colors[event])  # Assign unique color
            sizes.append(120)  # Larger size for "flashier" events
        else:
            colors.append("green")  # Green color for event 2956
            sizes.append(30)  # Normal size for event 2956

    fig, ax = plt.subplots(figsize=(13, 20))

    scatter = ax.scatter(temps_j, vitesses_j, c=colors, s=sizes, label='Points', alpha=0.9, edgecolor='k')

    # Add annotations for events other than 2956
    for i, event in enumerate(events_j):
        if event != 2956:
            ax.annotate(
                str(event),  # Text to display (event number)
                (temps_j[i], vitesses_j[i]),  # Position of the point
                textcoords="offset points",  # Text offset
                xytext=(5, 5),  # Pixel offset (x, y)
                fontsize=10,  # Font size
                color=event_colors[event]  # Text color based on event color
            )


    sorted_patches = [
        mpatches.Patch(color=event_colors[event], label=f"Event {event} ({count})")
        for event, count in sorted_events
    ]
    plt.legend(handles=sorted_patches, title="Events (sorted by frequency)", loc='upper left', bbox_to_anchor=(1.05, 1), fontsize=10)

    ax.set_title(f"Speed vs Time for Incident {incident_key}", fontsize=16)
    ax.set_xlabel("Time (seconds)", fontsize=12)
    ax.set_ylabel("Speed (km/h)", fontsize=12)

    ax.grid(True)


    plt.tight_layout()
    #plt.show()


### Plots of columns and their respective mean/occurence before the metric r 

In [ ]:

# Initializing the dictionary to store results for each incident type
dico = {}

# Loop over each unique incident type in the dataframe
for incident in df_paper["incident_type"].unique():
    dico[incident] = {
        "mean_of_mean_speed": 0,  # Placeholder for mean of mean speeds
        "mean_of_max_speed": 0,   # Placeholder for mean of maximum speeds
        "mean_of_ac_count": 0,    # Placeholder for mean of AC counts
        "mean_of_dc_count": 0,    # Placeholder for mean of DC counts
    }

    # Lists to accumulate the averages for each line in the dataframe
    mean_speeds = []
    max_speeds = []
    ac_counts = []
    dc_counts = []
    ac_count = 0 
    dc_count = 0
    
    # Loop over each row in the dataframe
    for _, row in df_paper.iterrows():
        secseq = row["filtered_seconde_sequence"]
      
        if row["incident_type"] == incident  :
            # Calculate values for the current row
           
            seq_kph = row["filtered_kph_sequence"]
            # Calculate the mean speed (if the list is not empty)
            
            ac_seq = row["filtered_ac_sequence"] 
            dc_seq = row["filtered_dc_sequence"] 
                
            # Filter the sequences for the desired time window (-600 to +600 seconds)
            zoom_kph = [kph for kph, sec in zip(seq_kph, secseq) if -300 <= sec <= 600]
            zoom_ac = [ac for ac, sec in zip(ac_seq, secseq) if -300 <= sec <= 600  ]
            zoom_dc = [dc for dc, sec in zip(dc_seq, secseq) if -300 <= sec <= 600  ]
            
            # Calculate statistics only if zoomed lists are not empty
            if zoom_kph:
                mean_speeds.append(np.mean(zoom_kph))
                max_speeds.append(np.max(zoom_kph))
            if zoom_ac:
                
                ac_counts.append(np.sum(zoom_ac) )  # Normalized AC count
            if zoom_dc:
                dc_counts.append(np.sum(zoom_dc) )  # Normalized 
    # Calculate global means for this incident type
    dico[incident]["mean_of_mean_speed"] = np.mean(mean_speeds)
    dico[incident]["mean_of_max_speed"] = np.mean(max_speeds)
    dico[incident]["mean_of_ac_count"] = np.mean(ac_counts)
    dico[incident]["mean_of_dc_count"] = np.mean(dc_counts)

# Print the dictionary containing the global means for each incident type
print(dico)





# Extract data from the dictionary
incident_types = [int(key) for key in dico.keys()]
mean_of_mean_speed = [dico[key]["mean_of_mean_speed"] for key in dico]
mean_of_max_speed = [dico[key]["mean_of_max_speed"] for key in dico]
mean_of_ac_count = [dico[key]["mean_of_ac_count"] for key in dico]
mean_of_dc_count = [dico[key]["mean_of_dc_count"] for key in dico]


plt.figure(figsize=(14, 8))
bar_width = 0.2  # Width of the bars in the bar chart
x = np.arange(len(incident_types))  # X-axis positions for each incident type

# Plot the bars for each statistic
plt.bar(x - 1.5 * bar_width, mean_of_mean_speed, bar_width, label="Mean of Mean Speed", color="blue", alpha=0.7)
plt.bar(x - 0.5 * bar_width, mean_of_max_speed, bar_width, label="Mean of Max Speed", color="orange", alpha=0.7)
plt.bar(x + 0.5 * bar_width, mean_of_ac_count, bar_width, label="Mean of AC Count", color="green", alpha=0.7)
plt.bar(x + 1.5 * bar_width, mean_of_dc_count, bar_width, label="Mean of DC Count", color="red", alpha=0.7)
plt.xlabel("Incident Types", fontsize=14)
plt.ylabel("Values", fontsize=14)
plt.title("Summary Statistics by Incident Type", fontsize=16)
plt.xticks(x, incident_types, fontsize=12, rotation=45) 
plt.legend(fontsize=12) 
plt.tight_layout()  


#plt.show()




{np.int64(4): {'mean_of_mean_speed': np.float64(9.814070035841807), 'mean_of_max_speed': np.float64(34.67944078947368), 'mean_of_ac_count': np.float64(1.6842105263157894), 'mean_of_dc_count': np.float64(32.80263157894737)}, np.int64(13): {'mean_of_mean_speed': np.float64(16.753644713662833), 'mean_of_max_speed': np.float64(46.125683279742766), 'mean_of_ac_count': np.float64(1.5659163987138263), 'mean_of_dc_count': np.float64(19.938906752411576)}, np.int64(14): {'mean_of_mean_speed': np.float64(20.74253466872586), 'mean_of_max_speed': np.float64(51.723208041958046), 'mean_of_ac_count': np.float64(5.265734265734266), 'mean_of_dc_count': np.float64(13.566433566433567)}, np.int64(2): {'mean_of_mean_speed': np.float64(7.348842541153902), 'mean_of_max_speed': np.float64(33.010451680672276), 'mean_of_ac_count': np.float64(1.2521008403361344), 'mean_of_dc_count': np.float64(57.21008403361345)}, np.int64(11): {'mean_of_mean_speed': np.float64(11.00263363642784), 'mean_of_max_speed': np.float64(

### Plots of columns and their respective mean/occurence after the metric r 

In [ ]:


# Initializing the dictionary to store results for each incident type
dico = {}
dico_false_counts = {}  # Dictionary for counts when AC and DC are both false

# Loop over each unique incident type in the dataframe
for incident in df_paper["incident_type"].unique():
    dico[incident] = {
        "mean_of_mean_speed": 0,  # Placeholder for mean of mean speeds
        "mean_of_max_speed": 0,   # Placeholder for mean of maximum speeds
        "mean_of_ac_count": 0,    # Placeholder for mean of AC counts
        "mean_of_dc_count": 0,    # Placeholder for mean of DC counts,
    }
    dico_false_counts[incident] = 0  # Initialize count for AC & DC both false

    # Lists to accumulate the averages for each line in the dataframe
    mean_speeds = []
    max_speeds = []
    ac_counts = []
    dc_counts = []
    
    # Loop over each row in the dataframe
    for _, row in df_paper.iterrows():
        secseq = row["filtered_seconds_metric_r"]
      
        if row["incident_type"] == incident:
            # Extract sequences for this row
            seq_kph = row["filtered_kph_metric_r"]
            ac_seq = row["filtered_ac_metric_r"]
            dc_seq = row["filtered_dc_metric_r"]
            
            # Filter the sequences for the desired time window (-300 to +600 seconds)
            zoom_kph = [kph for kph, sec in zip(seq_kph, secseq) if -300 <= sec <= 600]
            zoom_ac = [ac for ac, sec in zip(ac_seq, secseq) if -300 <= sec <= 600]
            zoom_dc = [dc for dc, sec in zip(dc_seq, secseq) if -300 <= sec <= 600]
            
            # Count the cases where AC and DC are both false
            ac_dc_false_count = sum(1 for ac, dc in zip(zoom_ac, zoom_dc) if not ac and not dc)
            dico_false_counts[incident] += ac_dc_false_count
            
            # Calculate statistics only if zoomed lists are not empty
            if zoom_kph:
                mean_speeds.append(np.mean(zoom_kph))
                max_speeds.append(np.max(zoom_kph))
            if zoom_ac:
                ac_counts.append( np.sum(zoom_ac))  # Normalized AC count
            if zoom_dc:
                dc_counts.append( np.sum(zoom_dc))  # Normalized DC count

    # Calculate global means for this incident type
    dico[incident]["mean_of_mean_speed"] = np.mean(mean_speeds) if mean_speeds else 0
    dico[incident]["mean_of_max_speed"] = np.mean(max_speeds) if max_speeds else 0
    dico[incident]["mean_of_ac_count"] = np.mean(ac_counts) if ac_counts else 0
    dico[incident]["mean_of_dc_count"] = np.mean(dc_counts) if dc_counts else 0

# Plot 1: Summary statistics by incident type
incident_types = [int(key) for key in dico.keys()]
mean_of_mean_speed = [dico[key]["mean_of_mean_speed"] for key in dico]
mean_of_max_speed = [dico[key]["mean_of_max_speed"] for key in dico]
mean_of_ac_count = [dico[key]["mean_of_ac_count"] for key in dico]
mean_of_dc_count = [dico[key]["mean_of_dc_count"] for key in dico]

plt.figure(figsize=(14, 8))
bar_width = 0.2
x = np.arange(len(incident_types))

# Plot the bars for each statistic
plt.bar(x - 1.5 * bar_width, mean_of_mean_speed, bar_width, label="Mean of Mean Speed", color="blue", alpha=0.7)
plt.bar(x - 0.5 * bar_width, mean_of_max_speed, bar_width, label="Mean of Max Speed", color="orange", alpha=0.7)
plt.bar(x + 0.5 * bar_width, mean_of_ac_count, bar_width, label="Mean of AC Count", color="green", alpha=0.7)
plt.bar(x + 1.5 * bar_width, mean_of_dc_count, bar_width, label="Mean of DC Count", color="red", alpha=0.7)

plt.xlabel("Incident Types", fontsize=14)
plt.ylabel("Values", fontsize=14)
plt.title("Summary Statistics by Incident Type", fontsize=16)
plt.xticks(x, incident_types, fontsize=12, rotation=45)
plt.legend(fontsize=12)
plt.tight_layout()
#plt.show()




### Plot when both DC & AC are False : Notice that both DC-AC = False  is really discriminative among classes 6 & 14 as observed in the plots ( before & after the metric r)above while others classes are not concerned with this

In [ ]:

# Plot 2: Count of AC and DC both false by incident type with spacing
false_counts = [dico_false_counts[key] for key in dico_false_counts]

# Create spaced x-axis positions for better readability
spaced_x = np.arange(len(incident_types)) * 2  # Multiply by 2 to create spacing

plt.figure(figsize=(12, 6))
plt.bar(spaced_x, false_counts, color="purple", alpha=0.7, width=0.5)
plt.xlabel("Incident Types", fontsize=14)
plt.ylabel("Count of AC and DC False", fontsize=14)
plt.title("Count of Cases Where AC and DC Are Both False by Incident Type", fontsize=16)
plt.xticks(spaced_x, incident_types, fontsize=12, rotation=45)
plt.tight_layout()
#plt.show()

# F1 Score & treshold retained

In [47]:
# Set an initial threshold tr
best_f1_score   = 0.9171
best_threshold  = 0.20
filtered_events = filter_events(relevance_metric, best_threshold)

print({key: value[:10] for key, value in filtered_events.items()})

{4: [1132, 2970, 2982, 2702, 1250, 2980, 3082, 3086, 1286, 1720], 13: [2744, 4148, 4394, 4396, 4124, 2956, 4068, 3636, 4066, 4078], 14: [4114, 4168, 4156, 2540, 4140, 3986, 2492, 2712, 4152, 4002], 2: [2708, 4016, 358, 4056, 4032, 4054, 4022, 3506, 4050, 4122], 11: [144, 1338, 2796, 2198, 2566, 2148, 2794, 2108, 2784, 200], 99: [4004, 4032, 2852, 4110, 2854, 4102, 4106, 2498, 4002, 2752], 9: [1572, 1608, 3792, 1798, 3782, 1786, 1790, 3794, 1782, 4076], 17: [990, 1250, 4130, 992, 1578, 2998, 3984, 3040, 2514, 2516], 3: [1802, 1808, 1822, 1828, 1872], 16: [], 6: [2712, 3082, 1780, 3084, 2714, 1138, 1054, 4190, 2934, 2320], 7: [728, 3424]}


### Additional Code for One-at-a-Time Procedure with unretained events : We didn't find any additional event not droping the F1score

In [44]:
# Additional Code for One-at-a-Time Procedure with unretained events
# Further refine by adding unretained events one at a time to see if they improve classifications
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
best_f1_score   = 0.9171
best_threshold  = 0.2

filtered_events = filter_events(relevance_metric, best_threshold)

def refine_with_unretained_events(df, relevance_metric, best_threshold, best_f1_score):
    # Filter relevant events based on the optimal threshold
    refined_filtered_events = filter_events(relevance_metric, best_threshold)
    unretained_events = {event for event_metrics in relevance_metric.values() for event, r in event_metrics.items() if r < best_threshold}

    # Extract existing events from the filtered sequences
    X = df["filtered_events_metric_r"]
    y = df['incident_type']
    unique_events = list(set(event for events in refined_filtered_events.values() for event in events))

    # List to track added events
    added_events = []

    # Using the One-at-a-time strategy: add one event at a time and reevaluate
    for event in unretained_events:
        # Add the event to the unique events list
        unique_events_with_event = unique_events + [event]
        
        # Vectorize the data with the added event
        X_vectorized = [
            [1 if e in seq else 0 for e in unique_events_with_event]
            for seq in X
        ]
        X_vectorized_df = pd.DataFrame(X_vectorized, columns=unique_events_with_event)

        # Update the column 'filtered_events_metric_r' with the current filtered events
        df['filtered_events_metric_r'] = df.apply(
            lambda row: filter_events_row_train(row, refined_filtered_events), axis=1
        )

        # Perform cross-validation with weighted F1-score
        stratified_kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        clf = RandomForestClassifier(n_estimators=500, random_state=42)
        
        # Calculate the weighted F1-score
        f1_scores = cross_val_score(clf, X_vectorized_df, y, cv=stratified_kf, scoring='f1_weighted')
        
        # If the F1-score improves, update the list
        f1_current = f1_scores.mean()
        if f1_current > 0.9171:
            print(f1_current)
            best_f1_score = f1_current
            unique_events.append(event)  # Add the event if the F1-score improves
            added_events.append(event)   # Add to the list of added events
            print(f"Event {event} added. New F1-score: {f1_current:.4f}")

    # Display 
    print("\nAdded events during refinement:")
    print(added_events)

    print(f"Final F1-score with unretained events: {best_f1_score}")
    return best_f1_score


#final_f1_score = refine_with_unretained_events(df_paper, relevance_metric, best_threshold, best_f1_score)


#filtered_events = filter_events(relevance_metric, best_threshold)
{key: value[:10] for key, value in filtered_events.items()}

{4: [1132, 2970, 2982, 2702, 1250, 2980, 3082, 3086, 1286, 1720],
 13: [2744, 4148, 4394, 4396, 4124, 2956, 4068, 3636, 4066, 4078],
 14: [4114, 4168, 4156, 2540, 4140, 3986, 2492, 2712, 4152, 4002],
 2: [2708, 4016, 358, 4056, 4032, 4054, 4022, 3506, 4050, 4122],
 11: [144, 1338, 2796, 2198, 2566, 2148, 2794, 2108, 2784, 200],
 99: [4004, 4032, 2852, 4110, 2854, 4102, 4106, 2498, 4002, 2752],
 9: [1572, 1608, 3792, 1798, 3782, 1786, 1790, 3794, 1782, 4076],
 17: [990, 1250, 4130, 992, 1578, 2998, 3984, 3040, 2514, 2516],
 3: [1802, 1808, 1822, 1828, 1872],
 16: [],
 6: [2712, 3082, 1780, 3084, 2714, 1138, 1054, 4190, 2934, 2320],
 7: [728, 3424]}

# LCSS

In [30]:

from itertools import combinations
from functools import reduce

# Algorithm to find the LCS between two sequences! Non-contiguous! BACKTRACKING VERSION
def lcs(seq1, seq2):
    # Create a 2D list (matrix) to store lengths of longest common subsequence at each position
    lengths = [[0] * (len(seq2) + 1) for _ in range(len(seq1) + 1)]
    
    # Fill the matrix with the length of the longest common subsequence between seq1 and seq2
    for i, x in enumerate(seq1):
        for j, y in enumerate(seq2):
            if x == y:
                lengths[i + 1][j + 1] = lengths[i][j] + 1  # If the elements match, increment the length
            else:
                lengths[i + 1][j + 1] = max(lengths[i + 1][j], lengths[i][j + 1])  # Else, take the max of the two possible directions
    
    # Backtracking to obtain the LCS sequence
    result = []
    x, y = len(seq1), len(seq2)
    while x != 0 and y != 0:
        if lengths[x][y] == lengths[x - 1][y]:
            x -= 1  # Move up if the value is the same as the one above
        elif lengths[x][y] == lengths[x][y - 1]:
            y -= 1  # Move left if the value is the same as the one to the left
        else:
            result.append(seq1[x - 1])  # If the values match, add the element to the result
            x -= 1
            y -= 1
    
    return result[::-1]  # Reverse the result list to get the LCS in correct order

def lcss_two_(seq1, seq2):
        n, m = len(seq1), len(seq2)
        dp = [[[] for _ in range(m + 1)] for _ in range(n + 1)]
        
        # Build the dynamic programming table to store LCSS results for each pair of indices
        for i in range(1, n + 1):
            for j in range(1, m + 1):
                if seq1[i - 1] == seq2[j - 1]:
                    dp[i][j] = dp[i - 1][j - 1] + [seq1[i - 1]]  # If elements match, extend the sequence
                else:
                    dp[i][j] = max(dp[i - 1][j], dp[i][j - 1], key=len)  # Otherwise, take the longer sequence from the top or left
        
        return dp[-1][-1]
def lcss_multiple(lists):  # DOUBLE FOR LOOP VERSION
    """
    Calculates the LCSS of multiple lists
    
    
    """
    # Function to calculate the LCSS of two lists
    if isinstance(lists, int )  : 
        return lists
    def lcss_two(seq1, seq2):
        n, m = len(seq1), len(seq2)
        dp = [[[] for _ in range(m + 1)] for _ in range(n + 1)]
        
        # Build the dynamic programming table to store LCSS results for each pair of indices
        for i in range(1, n + 1):
            for j in range(1, m + 1):
                if seq1[i - 1] == seq2[j - 1]:
                    dp[i][j] = dp[i - 1][j - 1] + [seq1[i - 1]]  # If elements match, extend the sequence
                else:
                    dp[i][j] = max(dp[i - 1][j], dp[i][j - 1], key=len)  # Otherwise, take the longer sequence from the top or left
        
        return dp[-1][-1]  # Return the longest common subsequence for the two lists
    
    # Compute the LCSS for multiple lists by iteratively combining two lists at a time
    if not lists:
        return []
    common_subsequence = lists[0]  # Start with the first list
    for lst in lists[1:]:
        common_subsequence = lcss_two(common_subsequence, lst)  # Update the common subsequence with each additional list
        if not common_subsequence:  # If no common subsequence is found, break early
            break
    
    return common_subsequence



# Define the filter function for events based on time range
def alpha():
    """This function extracts all rows from the dataframe corresponding to an incident cause containing only 
    events filtered by the metric 'r' and within the timestamp range [-144400, 600], 
    and places them in the dictionary: rows_for_each_incident_class with key 'incident_type'
    
    It returns:
    { 4 : [[ 2956, 2301, ...], [ 4122, ...], ...],
      17 : [[...], [...], ...]}
    """
    rows_for_each_incident_class = {}  # Dictionary to store filtered events
    
    # Filter events based on rows from the DataFrame
    for incident, events in filtered_events.items():
        
        # Retrieve the filtered events for this incident from the DataFrame
        lcss_filtered = df_paper[df_paper["incident_type"] == incident]['filtered_events_metric_r'].values
        
        # Apply event filtering for each window
        rows_for_each_incident_class[incident] = []
        
        # Only add non-empty lists to the dictionary
        for event_list in lcss_filtered:
            if len(event_list) >= 1:
                rows_for_each_incident_class[incident].append(event_list)

    return rows_for_each_incident_class


# Set the variable rows_for_each_incident_class
rows_for_each_incident_class = alpha()

dico_lcss_PAPER = {}
# Calculate the LCSS for each incident class
for incident, liste in rows_for_each_incident_class.items():
    dico_lcss_PAPER[incident] = lcss_multiple(rows_for_each_incident_class[incident])



# Events only occuring in one incident class. Then more generally events occuring in exactly n different classes

In [31]:
from collections import Counter


# Define windows for event filtering in different time ranges
windows_xk = [(-150.0, 150.0), (-300.0, 300.0), (-450.0, 450.0), (-600.0, 600.0), 
              (-900, 600), (-1200, 600), (-1800, 600), (-2400, 600), (-3000, 600), 
              (-4800, 600), (-6600, 600), (-13800, 600),(-14400,600)]  

# Function to calculate event frequencies for a window [-14400, 600]
def calculate_event_frequencies_14400_600_metric_r(df):
    """Count how many times an event (filtered by r + in [-14400,600]) appears in each incident class."""
    class_event_freq_14400_600 = {}
    # Loop over each incident type
    for incident_class in df['incident_type'].unique():
        class_data = df[df['incident_type'] == incident_class]
        # Count event occurrences in the filtered events for this incident class
        event_counter = Counter([event for seq in class_data["filtered_events_metric_r"] for event in seq]) 
        class_event_freq_14400_600[int(incident_class)] = event_counter
    
    return class_event_freq_14400_600

# Print the event frequencies for incident classes within the window
print(calculate_event_frequencies_14400_600_metric_r(df_paper))

# Main function to count events that occur exclusively in one class within a window
def counting_events_happening_only_in_ONE_class(i, window, df):
    """Main function that collects events which only occur in a single class."""
    # Step 1: Build a dictionary associating events with the classes where they appear
    event_to_classes = {}
    class_event_freq = calculate_event_frequencies_14400_600_metric_r(df)

    for incident_class, event_counts in class_event_freq.items():
        for event_id, count in event_counts.items():
            if event_id not in event_to_classes:
                event_to_classes[event_id] = set()
            event_to_classes[event_id].add(incident_class)
    
    # Step 2: Identify exclusive events for each class
    exclusive_events_per_class = {}

    for incident_class, event_counts in class_event_freq.items():
        exclusive_events = {}
        
        # Loop through the events of the class
        for event_id, count in event_counts.items():
            # Check if the event is exclusive to this class
            if len(event_to_classes[event_id]) == 1 and incident_class in event_to_classes[event_id]:
                # Check if the event occurs within the window
                events_list_of_sequences = df[df['incident_type'] == incident_class]["filtered_events_metric_r"].values
                seconds_list_of_sequences = df[df['incident_type'] == incident_class]["filtered_seconds_metric_r"].values
                
                is_event_in_window = False
                
                # Loop through event sequences and check for event occurrence in the defined time window
                for events_seq, sec_seq in zip(events_list_of_sequences, seconds_list_of_sequences):
                    for idx, event in enumerate(events_seq):
                        if event == event_id and window[i][0] <= sec_seq[idx] <= window[i][1]:
                            is_event_in_window = True
                            break
                    if is_event_in_window:
                        break
                
                # If the event is in the window, add it to the exclusive events dictionary
                if is_event_in_window:
                    exclusive_events[event_id] = count

        exclusive_events_per_class[incident_class] = exclusive_events

    return exclusive_events_per_class

# Calculate and print exclusive events for class 12 within the window
class_event_freq_14400_600 = calculate_event_frequencies_14400_600_metric_r(df_paper)
counting_events_happening_only_in_ONE_class_var_bewteen_14400_600 = counting_events_happening_only_in_ONE_class(12, windows_xk, df_paper)


# Function to count events that occur in exactly `n` classes within a window
def counting_events_happening_in_exactly_n_classes(i, window, df, n):
    """
    Counts events that occur in exactly `n` classes and are found within the specified window. 
    This function generalizes the previous one for cases where events appear in multiple classes.
    """
    # Step 1: Build a dictionary associating events with the classes where they appear
    event_to_classes = {}
    class_event_freq = calculate_event_frequencies_14400_600_metric_r(df)

    for incident_class, event_counts in class_event_freq.items():
        for event_id, count in event_counts.items():
            if event_id not in event_to_classes:
                event_to_classes[event_id] = set()
            event_to_classes[event_id].add(incident_class)

    # Step 2: Identify events that occur in exactly `n` classes
    events_per_class = {}

    for incident_class, event_counts in class_event_freq.items():
        filtered_events_ = {}
        # Loop through events of the class
        for event_id, count in event_counts.items():
            # Check if the event appears in exactly `n` classes
            if len(event_to_classes[event_id]) == n:
                # Check if the event occurs within the window
                events_list_of_sequences = df[df['incident_type'] == incident_class]["filtered_events_metric_r"].values
                seconds_list_of_sequences = df[df['incident_type'] == incident_class]["filtered_seconds_metric_r"].values
                
                is_event_in_window = False

                # Loop through event sequences to check for the event in the time window
                for events_seq, sec_seq in zip(events_list_of_sequences, seconds_list_of_sequences):
                    for idx, event in enumerate(events_seq):
                        if event == event_id and window[i][0] <= sec_seq[idx] <= window[i][1]:
                            is_event_in_window = True
                            break
                    if is_event_in_window:
                        break
                
                # If the event is in the window, add it to the filtered events dictionary
                if is_event_in_window:
                    filtered_events_[event_id] = count

        events_per_class[incident_class] = filtered_events_

    return events_per_class

# Example with events happening in exactly 2 class indicents
print(counting_events_happening_in_exactly_n_classes(0, windows_xk, df_paper, 2))
print(counting_events_happening_in_exactly_n_classes(12, windows_xk, df_paper, 2))

{4: Counter({2970: 115, 1286: 84, 2972: 61, 2980: 48, 1484: 40, 2982: 36, 2652: 32, 3260: 30, 1214: 25, 3144: 25, 2702: 23, 3080: 23, 1072: 23, 154: 23, 1250: 20, 1268: 19, 3032: 17, 1640: 17, 1660: 17, 1680: 17, 1700: 17, 1720: 16, 1740: 16, 1760: 16, 1052: 16, 1056: 16, 1780: 15, 1018: 15, 1002: 14, 1132: 13, 3082: 12, 1224: 11, 2992: 11, 1142: 11, 2988: 11, 3002: 11, 1044: 11, 2602: 10, 2690: 9, 2706: 9, 1252: 9, 2884: 9, 1860: 8, 1094: 8, 3036: 7, 3132: 7, 1800: 7, 1820: 7, 1840: 7, 2986: 7, 3102: 6, 150: 6, 3728: 6, 3720: 6, 3086: 5, 1084: 5, 1090: 5, 3138: 5, 1006: 5, 1024: 5, 1226: 4, 1028: 4, 1118: 4, 1106: 4, 1020: 4, 1134: 3, 2058: 3, 1068: 3, 3484: 3, 1058: 2, 1070: 2, 3062: 2, 3064: 2, 3054: 2, 3056: 2, 1144: 2, 2996: 2, 1124: 2, 1108: 2, 3052: 2, 1738: 2, 3074: 1, 3076: 1, 3078: 1, 840: 1, 1864: 1, 1120: 1, 2400: 1, 3438: 1, 3068: 1, 3070: 1, 3072: 1, 1034: 1, 2420: 1, 2430: 1, 3652: 1, 2694: 1, 1140: 1, 2984: 1, 2990: 1, 3664: 1, 4376: 1, 4222: 1, 2648: 1, 1216: 1, 3084: 

## DEFINITION OF THE WINDOWS WITH THEIR RESPECTIVE DICTIONARIES


In [32]:

from sklearn.naive_bayes import MultinomialNB

import ast

# Definition of the time windows
windows = [(-150.0, 150.0), (-300.0, 300.0), (-450.0, 450.0), (-600.0, 600.0), (-900, 600), (-1200, 600), 
           (-1800, 600), (-2400, 600), (-3000, 600), (-4800, 600), (-6600, 600), (-13800, 600), (-14400, 600)]


def creation_dictionary_events_related_to_seconds( df, incident_association = {}): 
    """This function simply creates a dictionary where the keys are incidents
    and the values are lists, where each list element is a tuple: (event, time) associated with an incident

    ---------> This gives the link between events and their occurrence times"""
    
    # Creating the dictionary associating incidents with event-time tuples
    for incident in df_paper["incident_type"].unique():

        events_with_times = (
            df[df['incident_type'] == incident][['events_sequence', 'seconds_to_incident_sequence']]
            .apply(lambda row: list(zip((row['events_sequence']), (row['seconds_to_incident_sequence']))), axis=1)
            .tolist()
        )
        incident_association[int(incident)] = events_with_times
    return incident_association



def non_relevant_events_filter(incident_association, windows, i, df_paper):
    """Filter events by window i and relevance metric r"""
    new_incident_association = {}  # Dictionary to store filtered events
    
    # Iterate through the incidents in the incident association dictionary
    for incident, events in incident_association.items():
        
        # Get the filtered events for this incident from the DataFrame
        filtered_df_events = df_paper[df_paper['incident_type'] == incident]['filtered_events_metric_r'].values
        filtered_df_events_set = set([item for sublist in filtered_df_events for item in sublist])  
        
        # Apply the filtering for each window
        new_incident_association[incident] = []
        
        for event_list in events:
            filtered_event_list = [
                (event, time) for event, time in event_list
                if event in filtered_df_events_set and windows[i][0] <= time <= windows[i][1]  # Filter by window bounds
            ]
            
            # Add the filtered list of events to the association
            new_incident_association[incident].append(filtered_event_list)
        
    
    
    return new_incident_association

## Set the dictionary created by the function creation_dictionary_events_related_to_seconds() 
## which is then filtered by the non_relevant_events_filter function                    

incident_association = non_relevant_events_filter(creation_dictionary_events_related_to_seconds(df_paper), windows, 12, df_paper)



parameter_window_i = 3

def windows_zooming(parameter_window_i, incident_association_training):
    """Not used yet, but who knows?"""
    # Retrieve the time range for the given window
    window_range = windows[parameter_window_i]  # Access the time range of the window at index 'i'
    filtered_incident_association = {}  # Dictionary for filtered incidents
    # Filter the events for each incident
    for incident, event_lists in incident_association_training.items():
        # Filtering the events based on the time window
        filtered_event_lists = []
        for sublist in event_lists:
            # Check that the sublist is not empty and contains (event, time) tuples
            if len(sublist) != 0 and all(isinstance(item, tuple) and len(item) == 2 for item in sublist):
                filtered_sublist = [
                    (event, time) for event, time in sublist if window_range[0] <= time <= window_range[1]
                ]
                filtered_event_lists.append(filtered_sublist)

        # Update the dictionary with the filtered events for each incident
        filtered_incident_association[incident] = filtered_event_lists
    
    return filtered_incident_association



def keeping_events_only( alpha_zooming):
    """
    Simplifies the incident_association dictionary by retaining only the event IDs 
    (first element of the tuple) for events in the given filtered_events_for_incident_only_events

    """
    incident_association_events_only = {}

    for incident, events in alpha_zooming.items():
        # Filter the events, keeping only the first element (event IDs)
        incident_association_events_only[incident] = [
            [event for event, _ in event_list]
            for event_list in events
        ]
    
    return incident_association_events_only


### Functions that determine if a subsequence of a sequence is only present in one class 

In [33]:

def matching_rows_(param_window_i, incident_window) : 
    "this function decide if a portion of the LCSS is in the window without order"
    matching_rows = set()
    target_events = []
    for event in incident_window: 
         target_events.append(event)
    target_events = set(target_events)
    for _, row in df_paper.iterrows():
        eventsseq = row["filtered_events_metric_r"]
        secseq = row["filtered_seconds_metric_r"]
        
    
        
        found_events = {}

        for i, (event, sec) in enumerate(zip(eventsseq, secseq)):
            if event in target_events:
                found_events[event] = (i, sec) 

            
            if len(found_events) == len(target_events):
                indices, seconds = zip(*found_events.values())
                
                # Vérify if all seconds are in the corresponding window
                if all(windows[param_window_i][0] <= second <= windows[param_window_i][1] for second in seconds):
                   
                    matching_rows.add(row["incident_type"])
                    break  

    return matching_rows



def matching_rows_with_order(param_window_i, incident_window):
    """
    This function checks if the exact order of the target sequence (non-continuous)
    is present in the event sequences for each incident_typewithin the specified time window
    """
    matching_classes = set()  # incident tyopes that match the criteria
    target_events = list(incident_window)  
    # iterate  trhough the dataframe
    for _, row in df_paper.iterrows():
        eventsseq = row["filtered_events_metric_r"]  # get the filtered sequence
        secseq = row["filtered_seconds_metric_r"]    # same but ofor seocnds
        incident_type = row["incident_type"]         # incident class

        # Initialisation of indexes in order to track the target sequenc e
        target_index = 0  
        event_indices = []  

        # 
        for i, (event, sec) in enumerate(zip(eventsseq, secseq)):
            if event == target_events[target_index]:
                # check if event is in the time window
                if windows[param_window_i][0] <= sec <= windows[param_window_i][1]: 
                    event_indices.append(i)  # memorise index
                    target_index += 1  #next event

                # if all targets found
                if target_index == len(target_events):
                    matching_classes.add(incident_type)  
                    break  

    return matching_classes




def lcss_in_window(window,lcss_window) : 
    lcss = lcss_two_(window, lcss_window)
    return lcss

def minimum_recognisable_seq_first_half(window, lcss_window) : 
    i =  1
    
    print(f"nous somme a la lcss {lcss_window[:10]}")
    if len(matching_rows_with_order(window,lcss_window[:i]) ) == 1 : 
        return lcss_window[:i]
    while len(matching_rows_with_order(window,lcss_window[:i]) )> 1 : 
        lcss_window_i = lcss_window[:i+1]
        print(lcss_window_i, i)
        matching_i =matching_rows_with_order(window, lcss_window_i)
        
        if len(matching_i) == 1 : 
            return lcss_window_i
        i+=1
        if i == len(lcss_window)+1  : 
    
            return False


def minimum_recognisable_seq_second_half(window, lcss_window):
    """
    Same but for second half
    """
   
    idx_slice = len(lcss_window) // 2  # start from midpoint of the sequence
    liste_ret = []  
    j = idx_slice + 1  
    if len(matching_rows_with_order(window, lcss_window[idx_slice:j]))== 1 : 
            return lcss_window[:i]
    while len(matching_rows_with_order(window, lcss_window[idx_slice:j])) > 1:
        lcss_window_j = lcss_window[idx_slice:j+1]  
        
        matching_j = matching_rows_with_order(window, lcss_window_j)  
        

        if len(matching_j) == 1:  
            return lcss_window_j 
        
        j += 1 
       
        if j > len(lcss_window)+2:  
            break

    return False





def minimum_recognisable_seq_general(window, lcss_window):
    """
    combination of the two functions above
    """

    # Step 1 : first half
    def minimum_recognisable_seq_first_half(window, lcss_window):
        i = 1
        
        if len(matching_rows_with_order(window, lcss_window[:i])) == 1:
            return lcss_window[:i], i
        while len(matching_rows_with_order(window, lcss_window[:i])) > 1:
            lcss_window_i = lcss_window[:i+1]
            print(lcss_window_i, i)
            matching_i = matching_rows_with_order(window, lcss_window_i)
            if len(matching_i) == 1:
                print(lcss_window_i)
                return lcss_window_i, i+1
            i += 1
            if i == len(lcss_window)+1:
                return False, i
        return False, i

    # Step 2 : second half
    def minimum_recognisable_seq_second_half(window, lcss_window, start_idx):
        idx_slice = start_idx 
        liste_ret = []
        j = idx_slice + 1
       
        while len(matching_rows_with_order(window, lcss_window[idx_slice:j])) > 1:
            lcss_window_j = lcss_window[idx_slice:j+1]
            matching_j = matching_rows_with_order(window, lcss_window_j)
            if len(matching_j) == 1:
                print(lcss_window_j)
                return lcss_window_j, j+1
            j += 1
            if j > len(lcss_window)+2:
                break
        return False, j

    
    found_sequences = []
    start_idx = 0

    
    while start_idx < len(lcss_window):
       
        first_half_result, new_start_idx = minimum_recognisable_seq_first_half(window, lcss_window[start_idx:])
        if first_half_result:
            found_sequences.append(first_half_result)
            start_idx += new_start_idx
            continue  # Continue from the index

        # second half
        second_half_result, new_start_idx = minimum_recognisable_seq_second_half(window, lcss_window, start_idx)
        if second_half_result:
            found_sequences.append(second_half_result)
            start_idx = new_start_idx
            continue

        #if no sequence found stop
        break
    if len(found_sequences)>0 : 

        return found_sequences
    return False



result = minimum_recognisable_seq_general(12, [1780, 1780, 1780, 2970])
print("Sous-séquences minimales reconnues :", result)
def is_subsequence(subseq, seq):
    """
    Vérifie si 'subseq' est une sous-séquence de 'seq'
    """
    it = iter(seq)  
    return all(item in it for item in subseq)
def filter_sequences(sequences):
    """
   Filter sequences inorder to only keep sequences not contained in other sequences
   Delete also odublons
    """
    unique_sequences = list(map(list, set(map(tuple, sequences))))
    
    filtered = []
    for seq in unique_sequences:
        if all(is_subsequence(other, seq) for other in unique_sequences if other != seq):
            filtered.append(seq)
    return filtered
 

[1780, 1780] 1
[1780, 1780, 1780] 2
[1780, 1780, 1780, 2970] 3
[1780, 1780, 1780, 2970]
Sous-séquences minimales reconnues : [[1780, 1780, 1780, 2970]]


### List of maximal length between n lists

In [34]:

def maximal_lengh_list(incident_association) :
    """return the maximal length'list among n lists  """
    lengh_incident_list ={}
    incident_association_filtered = non_relevant_events_filter(incident_association, windows, 12 , df_paper)
    for incident, listes in incident_association_filtered.items() : 
        taille = 1
        for liste in listes : 
           
            if len(liste) > taille :
                taille = len(liste)
      
           
        lengh_incident_list[incident] = taille
                
    return lengh_incident_list


incident_association =  non_relevant_events_filter(creation_dictionary_events_related_to_seconds(df_paper), windows, 12 , df_paper)

print(maximal_lengh_list(incident_association))


{4: 72, 13: 1312, 14: 335, 2: 214, 11: 422, 99: 395, 9: 90, 17: 21, 3: 152, 16: 1, 6: 1303, 7: 1}


### Functions that will process the "LLCSS" and determine all subsequences only occuring in just one class 

In [35]:


def LLCSS(df_paper, incident_association_training) :
    """This algorithm has two variant :  1.Consider windows
                                         2.Don't consider windows 
    'Choose your school of thought : Here we choose to consider windows ' 
    """

    """We establish in how many class(es) an event appears"""

    event_to_classes = {} 
    for _, row in df_paper.iterrows():
        inci = row["incident_type"]
        for event in row["filtered_events_metric_r"]:
            if event not in event_to_classes:
                event_to_classes[event] = set()
            event_to_classes[event].add(inci)
    

    big_liste_of_big_dico_LCSS_paper  ={}

    # initilization of the dictionnary containg the maximum lenghth  list of each incident
    dico_data = maximal_lengh_list(incident_association_training)

    # each window
    for parameter_window_i in range(13):
        big_dico_lcss_PAPER_by_window = {}
        alpha_zooming = windows_zooming(parameter_window_i, incident_association_training)
        
        # keeps evens only
        alpha_zooming_events_only = keeping_events_only(alpha_zooming)
        dico_lcss_PAPER_by_window = {}

        # Lcss for each incident class
        for incident, liste in alpha_zooming_events_only.items():
            dico_lcss_PAPER_by_window[incident] = []
            # each maximal list for each incident class
            for j in range(dico_data[incident]+1):
                
                # lcss of length minmim j 
                filtered_liste = [sublist for sublist in liste if len(sublist) > j]
                if filtered_liste:
                    # lcss of length at least j and add it to dico_lcss_PAPER_by_window 
                    if lcss_multiple(filtered_liste) : 
                        dico_lcss_PAPER_by_window[incident].append(lcss_multiple(filtered_liste))
                
            
        for incident, listes in dico_lcss_PAPER_by_window.items():
            for i in range(len(listes)):
                listes[i] = [
                    event for event in listes[i] if len(event_to_classes.get(event, [])) > 1 #  We are only interested in events occuring in more than one class
                                                                                             # Because we already have acces to events only occuring in one class
                                                                                             #Since if an event is only present in one class, the algorithm may 
                                                                                             # keep a sequence containing this particular event, however the others events
                                                                                             # won't yield any additional information 
                ]

        for incident, liste in dico_lcss_PAPER_by_window.items():
           
            if incident not in big_dico_lcss_PAPER_by_window:
                big_dico_lcss_PAPER_by_window[incident] = []
            for sub_liste in liste : 
                big_dico_lcss_PAPER_by_window[incident].append(sub_liste)
       
        
        for incident, listes in big_dico_lcss_PAPER_by_window.items() : 
                    if listes : 
                        
                        big_dico_lcss_PAPER_by_window[incident]=  filter_sequences(listes)

        big_liste_of_big_dico_LCSS_paper[parameter_window_i] = big_dico_lcss_PAPER_by_window
   
    for window, incidents in big_liste_of_big_dico_LCSS_paper.items():  # Parcourir chaque fenêtre
        for incident, listes in incidents.items():  # Parcourir chaque incident et ses listes
            
            filtered_listes_ = [minimum_recognisable_seq_general(12, liste ) for liste in listes if minimum_recognisable_seq_general(12, liste ) is not False]
            
            big_liste_of_big_dico_LCSS_paper[window][incident] = filtered_listes_
   
    return big_liste_of_big_dico_LCSS_paper
 
#big_dico_lcss_PAPER_by_window = LLCSS(df_paper, incident_association)
#print(big_dico_lcss_PAPER_by_window)

# Counting the total number of events for each incident class

In [36]:

from collections import defaultdict

# Calculation of prior probabilities p(cj)
def calculate_prior_probabilities(df):
    incident_counts = df['incident_type'].value_counts()
    total_incidents = len(df)
    priors = incident_counts / total_incidents
    return priors


priors = calculate_prior_probabilities(df)  # P-dictionary containing the probabilities of the classes cj's


from collections import defaultdict, Counter


def count_events_per_class(incident_association):
    """
    Counts how many times each event appears in the incidents
    
    example of return : {4: [(1132, 13), (2970, 115), (2982, 36), (2702, 23).... }
    """
    # Dictionary to store results
    class_event_counts = defaultdict(Counter)
    
    # For each class in the incident association
    for class_id, incidents in incident_association.items():
        # For each incident in this class
        for incident in incidents:
            # Extract the event without considering the second (only the first element of the tuple)
            events = [event[0] for event in incident]
            # Add the events to the Counter for this class
            class_event_counts[class_id].update(events)
    
    # Convert the Counter into a list of tuples (event, frequency) for each class
    class_event_counts = {class_id: list(events.items()) for class_id, events in class_event_counts.items()}
    
    return class_event_counts

print("count_events_per_class(incident_association)")
print(count_events_per_class(incident_association))


count_events_per_class(incident_association)
{4: [(1132, 13), (2970, 115), (2982, 36), (2702, 23), (1250, 20), (2980, 48), (3082, 12), (3086, 5), (1286, 84), (1720, 16), (1740, 16), (1760, 16), (1780, 15), (2652, 32), (3036, 7), (1134, 3), (3032, 17), (1640, 17), (1660, 17), (1680, 17), (1700, 17), (1214, 25), (3260, 30), (3132, 7), (1800, 7), (1820, 7), (1840, 7), (1860, 8), (2972, 61), (2690, 9), (2706, 9), (1052, 16), (3074, 1), (3076, 1), (3078, 1), (3080, 23), (1072, 23), (1058, 2), (1084, 5), (1070, 2), (1224, 11), (1090, 5), (840, 1), (1252, 9), (2992, 11), (1142, 11), (3144, 25), (3138, 5), (1864, 1), (1002, 14), (1120, 1), (2988, 11), (1226, 4), (1028, 4), (1018, 15), (2400, 1), (3438, 1), (2058, 3), (1006, 5), (1024, 5), (154, 23), (2986, 7), (3062, 2), (3064, 2), (3068, 1), (3070, 1), (3072, 1), (3054, 2), (3056, 2), (1068, 3), (1034, 1), (3002, 11), (2420, 1), (2430, 1), (3652, 1), (2694, 1), (1044, 11), (1094, 8), (1144, 2), (2884, 9), (1140, 1), (3102, 6), (1118, 4), (126

 # Calculation of conditional probabilities


In [37]:


# Calculation of prior probabilities p(cj) for each class (incident type)
priors_proba = {13: 0.31454006,
                 99: 0.17309594, 
                 14: 0.14737883,
                   2: 0.11770524,
                     9: 0.115727, 
                     4: 0.07715134, 
                     11: 0.02571711,
                     17: 0.0098912,
                       6: 0.00593472,
                       3: 0.0049456, 
                       16: 0.00395648,
                         7: 0.00395648}

"""Probabilities for each class after filtering events"""
priors_proba_filtered = {13: 0.10113960113960115, 
                         99: 0.2264957264957265, 
                         14: 0.16951566951566951,
                         2: 0.07407407407407407,
                         9: 0.1225071225071225,
                         4: 0.14245014245014245,
                         11: 0.11253561253561253,
                         17: 0.007122507122507123,
                         6: 0.02849002849002849, 
                         3: 0.005698005698005698,
                         16: 0.007122507122507123,
                         7: 0.002849002849002849}

# Count occurrences of each incident type in the "incident_type" column
def transform_event(dico):
    new_dico = {}
    for incident, liste in dico.items():
        # Initialize the list for the key if it doesn't exist
        if incident not in new_dico:
            new_dico[incident] = []
        # Add the first elements of the tuples to the list
        for tuples in liste:
            new_dico[incident].append(tuples[0])
    return new_dico


def transform_count(dico):
    new_dico = {}
    for incident, liste in dico.items():
        # Initialize the list for the key if it doesn't exist
        if incident not in new_dico:
            new_dico[incident] = []
        # Add the second elements (counts) of the tuples to the list
        for tuples in liste:
            new_dico[incident].append(tuples[1])
    return new_dico

# Example event windows for testing
win = [2982, 3082, 3086, 1286, 1720, 1740, 1760]
win2 = [2956, 2956, 2956 ]
win3 = [2982, 3082, 3086, 1286, 1720, 1740, 1760, 1780]


# Function to calculate the conditional probability P(event | class)
def conditional_probability(event, incident, incident_window, training_incident_association, beta= 0.01):
    # Get the count of events in the given class
    #If the window around an incident is empty----> This is probably useless since later in the cascaded classifier, only windows of positive size are considered
    # even though this will probably lead to incident with precision =  0 = recall  
    if incident_window == [] :
        return pow(beta, 2)
    occurence_of_events_in_the_class_i = count_events_per_class(training_incident_association)
    dico_events = transform_event(occurence_of_events_in_the_class_i)
    dico_counts = transform_count(occurence_of_events_in_the_class_i)
    boolean_window_lenght_one = False
    #We may encounter a problem not talked during our presentation ( discovered 2 days later. cf mail) that is when the number of features is only one
    # ( repetitions taken into acount) that may also lead to the wrong classification of the class with the highest prior probability that is our case class 13
    if len(set(incident_window)) == 1 : 
        boolean_window_lenght_one = True
    # Initialize count for the occurrence of event in the given class
    card_xki = 0
    if incident in dico_events:
        if event in dico_events[incident]:
            index_ =  dico_events[incident].index(event)
            card_xki += dico_counts[incident][index_]
                
    # Calculate numerator
    num = card_xki + beta 
    if  boolean_window_lenght_one and num !=beta:  
        #We ensure that the number returned is less than 1 
        return card_xki/1000
    # Number of features (events) in the window
    terme_en_beta = beta * len(set(incident_window))
    
    # Initialize denominator for the probability calculation
    card_sum_xi = 0
    for event in set(incident_window): 
        if incident in dico_events: 
            if event in dico_events[incident]: 
                index_ = dico_events[incident].index(event)
                card_sum_xi += dico_counts[incident][index_]
    
    # Denominator for the probability
    denom = terme_en_beta + card_sum_xi
   
    #if we encounter the issue where the denominator is only function of beta i.e, when  sum [card(xki|cj) ] = 0
    if denom == terme_en_beta: 
        return pow(beta, 2)
  
    if denom != 0:
        # Return the conditional probability
        p_x_given_c = num / denom
        return p_x_given_c 

    # If the incident window is empty, force a default probability (in this case, class 13)
    return 0


# Original Naive Bayes

In [38]:
from itertools import combinations

# The lcss dictionnary obtained if we consider the lcss for each incident !irrespective of the window! though we didn't really obtained many lcss if we considered the windows
# -----> This is from where the idea came : considering LCSS of minimul length from 1 to n i.e, LLCSS : Locally LCSS, cf. section above

dico_lcss_PAPER = {4: [], 13: [], 14: [], 2: [], 11: [], 99: [], 9: [], 17: [],
                   3: [1802, 1808, 1822, 1828, 1802, 1808, 1822, 1828, 1802, 1808, 1802, 1808, 1822, 1828, 
                       1822, 1828, 1802, 1808, 1802, 1808, 1802, 1802, 1808, 1822, 1828, 1802, 1808, 1822, 
                       1828, 1802, 1808, 1822, 1828, 1802, 1808, 1822, 1828, 1802, 1808, 1822, 1828, 1822,
                         1828, 1802, 1808, 1802, 1822, 1808, 1828, 1802, 1822, 1808, 1822, 1828, 1822, 1828, 
                         1802, 1808, 1802, 1822, 1808, 1828, 1802, 1822, 1808, 1828, 1802, 1808, 1822, 1828,
                           1822, 1828, 1822, 1828, 1802, 1808, 1802, 1808, 1802, 1808, 1822, 1828, 1822, 1828,                              
                             1802, 1822, 1808, 1828, 1802, 1808, 1822, 1828, 1822, 1828, 1802, 1808, 1802, 1808,
                               1822, 1828, 1822, 1828, 1802, 1822, 1808, 1828, 1802, 1822, 1808, 1828, 1802, 1808,
                                 1822, 1828, 1802, 1808, 1822, 1828, 1822, 1828, 1802, 1808, 1822, 1828, 1802, 1808, 
                                 1822, 1828, 1802, 1808, 1822, 1828, 1802, 1808, 1822, 1828, 1802, 1822, 1808, 1828,
                                   1802, 1808, 1822, 1828, 1802, 1808, 1822, 1828, 1822, 1828], 
                      16: [2498, 3688, 3680, 3680, 3688, 3800, 3808, 3840, 3848, 2498], 6: [3082], 7: []}


(AUGMENTER LE THRESHOD à 0.3-0.5 afin de vérifier si des élémrents apparaissant dans  2 classe différentes exactement peuvent être discriminé) Des trucs qu'on faisait avant mais plus d'actualirté

In [39]:
def is_subsequence(subsequence, sequence):
    """
    Verifies if `subsequence` is a subsequence of `sequence` ; subsequence does not need to be contiguous !but must respect the order of elements!
    
    """
    it = iter(sequence)  # Create an iterator for the main sequence
    return all(item in it for item in subsequence)  # Check if each item of subsequence appears in order

# example
print(is_subsequence([1802, 1822, 1808], [1802, 777, 1808, 8888, 45, 1828]), "true or false")

def extract_k_subsequences(lst, k):
    """
    Extracts all contiguous subsequences of length k from a list

    
    """
    if k > len(lst):
        return []  # Cannot extract a subsequence longer than the list

    return [lst[i:i + k] for i in range(len(lst) - k + 1)]  # Return all contiguous subsequences of length k


def subsequence_at_least_k(subsequence, sequence, k):
    """
    Checks if at least `k` elements of `subsequence` appear in `sequence`, in order 

    """
    match_count = 0  
    seq_idx = 0  # Index to traverse the main sequence

    for elem in subsequence:
        while seq_idx < len(sequence) and sequence[seq_idx] != elem:
            seq_idx += 1  # Move to the next element in the main sequence
        if seq_idx < len(sequence):
            match_count += 1  # Increment match count if element is found
            seq_idx += 1  # Move to the next element in sequence
            if match_count >= k:  # If at least k matches are found, return True
                return True
    return False  # Return False if fewer than k matches are found


def is_lcss_in_list(lcss, window, limit):
    """
    This function checks if the Longest Common Subsequence (LCSS) appears in a given window 
    It counts how many LCSS events appear in the window and checks if they appear at least 7 times

    """
    count = 0  # Counter for the number of LCSS events found in the window
    for event in window:
        if event in lcss:  # Check if the event is part of the LCSS
            count += 1  # Increment count if event is part of LCSS
    if count >= limit:  # If 7 or more events from LCSS are found in the window, return True
        return limit
    return count



# Example of event occurrences in multiple classes
events_occuring_exactly2_times_in_classes = {4: {1132: 13, 2970: 115, 1250: 20, 3082: 12, 1214: 25, 3260: 30, 1840: 7, 2690: 9, 1052: 16, 1142: 11, 3144: 25, 3138: 5, 1864: 1, 1002: 14, 1226: 4, 2694: 1, 1094: 8, 1144: 2, 2884: 9, 2602: 10, 3728: 6, 3720: 6, 3084: 1, 3128: 1, 1122: 1, 1738: 2, 3012: 1, 3014: 1, 3018: 1, 3020: 1, 3022: 1, 824: 1, 1484: 40, 3108: 1}}
def highest_proba_ratio(incident_window, events_occuring_exactly2_times_in_classes, df_paper):
    """ 
    Main function: In the first phase, we check if for the same threshold of 0.2,
    for events that appear exactly in two different classes, we can distinguish them 
    with a ratio of 0.9. For example, if event 4120 appears 9 times in class 2 and only 
    once in class 99, it may be possible to distinguish these classes. 
    If not, we increase the threshold to 0.3 and proceed similarly, always looking for a ratio of 0.9.
    (This function won't be used actually )
    """
    
    count_total_of_both_incident1 = 0  # This will store the total count of events for incident 1
    count_total_of_both_incident2 = 0  # This will store the total count of events for incident 2
    dico_count_event_per_incident1 = {}  # Dictionary to store event counts for incident 1
    dico_count_event_per_incident2 = {}  # Dictionary to store event counts for incident 2

    # Iterate over unique events in the incident window
    for event in set(incident_window):
        # Iterate over incidents and event frequencies in events_occuring_exactly2_times_in_classes
        for incident, event_freq in events_occuring_exactly2_times_in_classes.items():
            count_incident_event1 = 0  # Counter for events in incident 1
            count_incident_event2 = 0  # Counter for events in incident 2
            dico_count_event_per_incident1[incident] = {}
            dico_count_event_per_incident2[incident] = {}

            # Check if the event exists in the incidents' event frequency
            if event in events_occuring_exactly2_times_in_classes[incident]:
                # Iterate through rows in df_paper (dataframe with event information)
                for _, row in df_paper.iterrows():
                    events_seq3 = row["filtered_events_metric_r"]  # Events in metric 3 (threshold 0.2)
                    #events_seq4 = row["filtered_events_metric_r_higher_threshold"]  # Events in metric 4 (threshold 0.3)
                    sec_seq = row["seconds_to_incident_sequence"]  # Time sequence to the incident

                    # Count occurrences of the event in events_seq3 if it matches the incident
                    for i, event_inseq_3 in enumerate(events_seq3):
                        if event_inseq_3 == event and row["incident_type"] == incident:
                            count_incident_event1 += 1

                    # Store the count of the event in incident 1
                    dico_count_event_per_incident1[incident][event] = count_incident_event1

    
    # Calculate the total ratio for incidents based on dico_count_event_per_incident1
    for incident, eventcount in dico_count_event_per_incident1.items():
        if eventcount != {}:
            for event, count in eventcount.items():
                count_total_of_both_incident1 += count

    # Normalize the event counts for incident 1 and check if the ratio is above 0.9
    for incident, event_count in dico_count_event_per_incident1.items():
        for event in event_count:
            if count_total_of_both_incident1 != 0:
                dico_count_event_per_incident1[incident][event] = dico_count_event_per_incident1[incident][event] / count_total_of_both_incident1

                if dico_count_event_per_incident1[incident][event] >= 0.9:
                    print(event, incident)  # Print the event and incident when the ratio exceeds 0.9
                    return incident, 1  # Return the incident and a value indicating high probability
    
    
    """
    for incident, eventcount in dico_count_event_per_incident2.items():
        if eventcount != {}:
            for event, count in eventcount.items():
                count_total_of_both_incident2 += count

    for incident, event_count in dico_count_event_per_incident2.items():
        for event in event_count:
            if count_total_of_both_incident2 != 0:
                dico_count_event_per_incident2[incident][event] = dico_count_event_per_incident2[incident][event] / count_total_of_both_incident2

                if dico_count_event_per_incident2[incident][event] >= 0.96:
                    print(event, incident)
                    return incident, 1
    """


def is_ordered_subsequence(subsequence, main_sequence):
    """
    verify if a non continuous subsequence is present in a sequence
    """
    it = iter(main_sequence)  
    return all(event in it for event in subsequence)
big_dico_lcss_PAPER_by_window_i_test = {0: {4: [[1286], [1800], [3082], [1680], [1820], [1700], [1840], [1720], [3132], [3260], [1216], [1860], [1740], [3032], [3036], [2652], [1760], [1640], [1002, 1780], [1018], [1660]], 13: [], 14: [], 2: [], 11: [], 99: [[1666, 260], [1670, 1806, 1810, 1686], [1690, 1826], [2596], [1830, 1706], [1710, 1846], [2490, 1850]], 9: [[1722], [1730], [1726], [1728]], 17: [], 3: [], 16: [], 6: [[1000, 2088, 2090], [4204], [1688]], 7: []}, 1: {4: [[1700], [1286], [1640], [1680], [3260], [3032], [1660], [2652]], 13: [], 14: [], 2: [], 11: [], 99: [], 9: [[3782], [3622], [3722], [4076], [3762], [1620], [3742]], 17: [[992]], 3: [], 16: [], 6: [], 7: []}, 2: {4: [], 13: [], 14: [], 2: [], 11: [], 99: [], 9: [[3782], [3622], [3722], [4076], [3762], [1620], [3742]], 17: [[992], [4130, 1250]], 3: [], 16: [], 6: [[2048, 2320], [1682], [1688], [2088, 2090]], 7: []}, 3: {4: [], 13: [], 14: [], 2: [], 11: [], 99: [], 9: [], 17: [[992], [4130, 1250]], 3: [], 16: [], 6: [[2048, 3082], [2320], [1682], [1688], [2088, 2090], [942, 1200], [1202], [188]], 7: []}, 4: {4: [], 13: [], 14: [], 2: [], 11: [], 99: [], 9: [], 17: [[992], [4130, 1250]], 3: [], 16: [], 6: [[2048, 3082], [4368], [2320], [1682], [1688], [4382], [2088, 2090], [942, 1200], [1202], [188]], 7: []}, 5: {4: [], 13: [], 14: [], 2: [], 11: [], 99: [], 9: [], 17: [[992], [4130, 1250]], 3: [], 16: [], 6: [[2048, 3082], [4368], [2320], [1682], [1688], [4382], [2088, 2090], [942, 1200], [1202], [188]], 7: []}, 6: {4: [], 13: [], 14: [], 2: [], 11: [], 99: [], 9: [], 17: [[992], [4130, 1250]], 3: [], 16: [], 6: [[2048, 3082], [4368], [2320], [1682], [1688], [4382], [2088, 2090], [942, 1200], [1202], [188]], 7: []}, 7: {4: [], 13: [], 14: [], 2: [], 11: [], 99: [], 9: [], 17: [[992], [4130, 1250]], 3: [], 16: [], 6: [[2048, 3082], [4368], [2320], [1682], [1688], [4382], [2088, 2090], [942, 1200], [1202], [188]], 7: []}, 8: {4: [], 13: [], 14: [], 2: [], 11: [], 99: [], 9: [], 17: [[992], [4130, 1250]], 3: [], 16: [], 6: [[2048, 3082], [4368], [2320], [1682], [1688], [4382], [2088, 2090], [942, 1200], [1202], [188]], 7: []}, 9: {4: [], 13: [], 14: [], 2: [], 11: [], 99: [], 9: [], 17: [[992], [4130, 1250]], 3: [], 16: [], 6: [[2048, 3082], [4368], [2320], [1682], [1688], [4382], [2088, 2090], [942, 1200], [1202], [188]], 7: []}, 10: {4: [], 13: [], 14: [], 2: [], 11: [], 99: [], 9: [], 17: [], 3: [[1828], [1802], [1808]], 16: [], 6: [[2048, 3082], [4368], [2320], [1682], [1688], [4382], [2088, 2090], [942, 1200], [1202], [188]], 7: []}, 11: {4: [], 13: [], 14: [], 2: [], 11: [], 99: [], 9: [], 17: [[992], [4130, 1578]], 3: [[1828], [1802], [1808]], 16: [], 6: [[2048, 3082], [2320], [4368], [1682], [1688], [4248], [4382], [2088, 2090], [942, 1200], [1202], [188], [964, 1100], [4314, 4316, 2654], [4318, 4190], [1000, 4204], [1138], [2934], [2040, 4222]], 7: []}, 12: {4: [], 13: [], 14: [], 2: [], 11: [], 99: [], 9: [], 17: [[992], [4130, 1578]], 3: [[1828], [1802], [1808]], 16: [], 6: [[2048, 3082], [2320], [4368], [1682], [1688], [4248], [4382], [2088, 2090], [942, 1200], [1202], [188], [964, 1100], [4314, 4316, 4190], [2654], [4318, 1000, 4204], [1138], [2934], [2040, 4222]], 7: []}}

def original_naive_bayes(  incident_window, incident_association, 
                         priors_proba_plis, counting_events_happening_only_in_ONE_class_var_bewteen_14400_600, beta=0.01):
    """Main function of this cell"""
    # STEP 1: Directly classify events occurring only in one class (counting_events_happening_only_in_ONE_class_var_bewteen_14400_600)
    
    for incident, event_seq in counting_events_happening_only_in_ONE_class_var_bewteen_14400_600.items():
        if counting_events_happening_only_in_ONE_class_var_bewteen_14400_600[incident] != []:  # Check if the event list for the incident is not empty
            for event in incident_window:
                    
                if event in counting_events_happening_only_in_ONE_class_var_bewteen_14400_600[incident]:  # Check if the event is in the sequence
                    # Occurence above a certain treshold ( around 100) seems to decrease dramatically the F1 score, since some events only occuring in class 13
                    # are present more than 1000 times in this class in the train set. Thus there is a very small probability that these events will only occur in class 13 in the test set 
                    if incident == 13 and counting_events_happening_only_in_ONE_class_var_bewteen_14400_600[13][event] <= 95: 

                        print(f"STAGE 1: {event} in class {incident}")
                        return incident, 1  # Return the corresponding class if event found in the sequence"""
                    elif incident != 13   : 
                        print(f"STAGE 1: {event} in class {incident}")
                        return incident, 1  
                    
    # Step 2 If None of the above is valid, then we aplly the Paper's Original Naive Baye* Classifier approach. 
    # *Of course, as discussed in the presentation, we refer here to the 
    # classifier slightly modified to take into account windows containing only events appearing in one Class --> The modification is in the conditional probabilities cell
    
    product_probs_by_class = {}
    # Calculate the conditional probabilities for each class using the Naive Bayes formula
    for incident, prior in priors_proba_plis.items():  # Loop through each class and its prior probability
        produit = 1
        for event in set(incident_window):  # Loop through each unique event in the incident window
            prob = conditional_probability(event, incident, incident_window, incident_association)  # Get conditional probability of the event for the given class
            produit *= prob  # Multiply the probabilities of all events in the window
        
        # Multiply the prior probability of the class by the product of the event probabilities
        produit = priors_proba_plis[incident] * produit
        # Store the result in the dictionary for the class
        product_probs_by_class[incident] = produit

    # Get the predicted class as the one with the highest probability
    predicted_class = max(product_probs_by_class, key=product_probs_by_class.get)
    max_proba = max(product_probs_by_class.values())  # Get the maximum probability
    print(predicted_class, max_proba) 
    return predicted_class, max_proba  # Return the predicted class and its probability

# Test the Naive Bayes classifier
print("\nNaive Bayes test ")
counting_events_happening_only_in_ONE_class_var = counting_events_happening_only_in_ONE_class(12, windows_xk, df_paper)
print(original_naive_bayes(  [1132], incident_association, priors_proba_filtered, counting_events_happening_only_in_ONE_class_var), "class")




False true or false

Naive Bayes test 
14 0.0018646723646723645
(14, 0.0018646723646723645) class


## Prequisites for the CASCADE CLASSIFIER

In [40]:
extracted_events_by_incident = {}
"""CONFIGURATIONS of the cascading windows """
conf = [[] for i in range(len(windows_xk)+1)]  # Initialize the list of configurations
for i in range(len(windows_xk)): 
        conf[i] = windows_xk[:i+1]  # For each window, add configurations from the start to the current window
windows_xk = [(-150.0, 150.0), (-300.0, 300.0), (-450.0, 450.0), (-600.0, 600.0), 
              (-900, 600), (-1200, 600), (-1800, 600), (-2400, 600), (-3000, 600), 
              (-4800, 600), (-6600, 600), (-13800, 600), (-14400,600)]  # Define the time windows



def compute_f1_score(y_true, y_pred):
    """
    Calculate the F1 score between the true values and the predicted values
    
    :param y_true: List or array of true labels (real classes)
    :param y_pred: List or array of predicted labels
    :return: The F1 score
    """
    return f1_score(y_true, y_pred, average='weighted')  # Calculate weighted F1 score

# Function to calculate the class priors (probabilities of each class) from the training set
def calculate_priors(training_set):
    class_counts = training_set['incident_type'].value_counts()  # Get the frequency of each incident type
    total_instances = len(training_set)  # Get the total number of incidents
    priors = {cls: count / total_instances for cls, count in class_counts.items()}  # Calculate priors
    return priors

from collections import defaultdict

priorssss = calculate_priors(df_paper)  # Calculate priors for the given DataFrame


def non_relevant_events_filter_windows_i(incident_association, windows, i, df_paper):
    """Filter events by window i and relevance metric r"""
    new_incident_association = {}  # Dictionary to store filtered events
    
    # Iterate through the incidents in the incident association dictionary
    for incident, events in incident_association.items():
        
        # Get the filtered events for this incident from the DataFrame
        filtered_df_events = df_paper[df_paper['incident_type'] == incident]['filtered_events_metric_r'].values
        filtered_df_events_set = set([item for sublist in filtered_df_events for item in sublist])  # Flatten and convert to set
        
        # Apply the filtering for each window
        new_incident_association[incident] = []
        
        for event_list in events:
            filtered_event_list = [
                (event, time) for event, time in event_list
                if event in filtered_df_events_set and windows[i][0] <= time <= windows[i][1]  # Filter by window bounds
            ]
            
            # Add the filtered list of events to the association
            new_incident_association[incident].append(filtered_event_list)
        
        # Debugging: Check the dictionary after filtering
    
    return new_incident_association


# Function to calculate probability priors for the training set
def probas_priors_training_test(dico):
    freq_ = {}  # Dictionary to store frequencies of incidents
    total_count = 0  # Total count of events across all incidents
    for incident, liste in dico.items(): 
        count_ = 0
        for event, count in liste: 
            count_ += count  # Count events for the incident
        total_count += count_  # Add to the total count
        freq_[incident] = count_  # Store frequency for the incident
    # Calculate the probability for each incident based on frequency
    for incident, freq in freq_.items():
        freq_[incident] = freq_[incident] / total_count  # Normalize the frequency to get the probability
    return freq_

print(probas_priors_training_test(count_events_per_class(non_relevant_events_filter_windows_i(incident_association, windows_xk, 12, df_paper))))  # Calculate priors for events
print(conf)  
conf = conf[:-1]  # Remove the last configuration
print(conf) 

#The 13 windows configuration
[[(-150.0, 150.0)], [(-150.0, 150.0), (-300.0, 300.0)], [(-150.0, 150.0), (-300.0, 300.0), (-450.0, 450.0)], [(-150.0, 150.0), (-300.0, 300.0), (-450.0, 450.0), (-600.0, 600.0)], [(-150.0, 150.0), (-300.0, 300.0), (-450.0, 450.0), (-600.0, 600.0), (-900, 600)], [(-150.0, 150.0), (-300.0, 300.0), (-450.0, 450.0), (-600.0, 600.0), (-900, 600), (-1200, 600)], [(-150.0, 150.0), (-300.0, 300.0), (-450.0, 450.0), (-600.0, 600.0), (-900, 600), (-1200, 600), (-1800, 600)], [(-150.0, 150.0), (-300.0, 300.0), (-450.0, 450.0), (-600.0, 600.0), (-900, 600), (-1200, 600), (-1800, 600), (-2400, 600)], [(-150.0, 150.0), (-300.0, 300.0), (-450.0, 450.0), (-600.0, 600.0), (-900, 600), (-1200, 600), (-1800, 600), (-2400, 600), (-3000, 600)], [(-150.0, 150.0), (-300.0, 300.0), (-450.0, 450.0), (-600.0, 600.0), (-900, 600), (-1200, 600), (-1800, 600), (-2400, 600), (-3000, 600), (-4800, 600)], [(-150.0, 150.0), (-300.0, 300.0), (-450.0, 450.0), (-600.0, 600.0), (-900, 600), (-1200, 600), (-1800, 600), (-2400, 600), (-3000, 600), (-4800, 600), (-6600, 600)], [(-150.0, 150.0), (-300.0, 300.0), (-450.0, 450.0), (-600.0, 600.0), (-900, 600), (-1200, 600), (-1800, 600), (-2400, 600), (-3000, 600), (-4800, 600), (-6600, 600), (-13800, 600)], [(-150.0, 150.0), (-300.0, 300.0), (-450.0, 450.0), (-600.0, 600.0), (-900, 600), (-1200, 600), (-1800, 600), (-2400, 600), (-3000, 600), (-4800, 600), (-6600, 600), (-13800, 600), (-14400, 600)]]


def non_relevant_events_filter_r_train(incident_association, windows, i, df_train):
                """Filter events by window i and relevance metric r"""
                new_incident_association = {}  # Dictionary to store filtered events
                
                # Iterate through the incidents in the incident association dictionary
                for incident, events in incident_association.items():
                    
                    # Get the filtered events for this incident from the DataFrame
                    filtered_df_events = df_train[df_train['incident_type'] == incident]['filtered_events_metric_r_train'].values
                    filtered_df_events_set = set([item for sublist in filtered_df_events for item in sublist])  # Flatten and convert to set
                    
                    # Apply the filtering for each window
                    new_incident_association[incident] = []
                    
                    for event_list in events:
                        filtered_event_list = [
                            (event, time) for event, time in event_list
                            if event in filtered_df_events_set and windows[i][0] <= time <= windows[i][1]  # Filter by window bounds
                        ]
                        
                        # Add the filtered list of events to the association
                        new_incident_association[incident].append(filtered_event_list)
                
                return new_incident_association
def windows_zooming_i(window,secseq, i ) : 
                """Consider only events in the timestamp of the window i """
                window_zoom_i =[ event for event, sec in zip(window, secseq) if  windows[i][0] <=sec <= windows[i][1]]
                return window_zoom_i


{4: 0.00756118464908104, 13: 0.8493730755801034, 14: 0.04899850807859569, 2: 0.04505602641018316, 11: 0.0051106243849792085, 99: 0.026232422308986446, 9: 0.007846871726502239, 17: 0.0004697965273148589, 3: 0.0009649874615116021, 16: 0.0, 6: 0.008373805669301336, 7: 1.2697203440942133e-05}
[[(-150.0, 150.0)], [(-150.0, 150.0), (-300.0, 300.0)], [(-150.0, 150.0), (-300.0, 300.0), (-450.0, 450.0)], [(-150.0, 150.0), (-300.0, 300.0), (-450.0, 450.0), (-600.0, 600.0)], [(-150.0, 150.0), (-300.0, 300.0), (-450.0, 450.0), (-600.0, 600.0), (-900, 600)], [(-150.0, 150.0), (-300.0, 300.0), (-450.0, 450.0), (-600.0, 600.0), (-900, 600), (-1200, 600)], [(-150.0, 150.0), (-300.0, 300.0), (-450.0, 450.0), (-600.0, 600.0), (-900, 600), (-1200, 600), (-1800, 600)], [(-150.0, 150.0), (-300.0, 300.0), (-450.0, 450.0), (-600.0, 600.0), (-900, 600), (-1200, 600), (-1800, 600), (-2400, 600)], [(-150.0, 150.0), (-300.0, 300.0), (-450.0, 450.0), (-600.0, 600.0), (-900, 600), (-1200, 600), (-1800, 600), (-240

#                                              Cascaded Classifier : Ensemble approach

### Future work could possibly incorporate AC-DC states into the Original Naive Bayes Formula

In [41]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score



def cascaded_naive_bayes_classifier_with_configs(df, conf, threshold=0.0005, n_splits=10):
    from collections import defaultdict
    from sklearn.model_selection import StratifiedKFold


   
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    config_results = {}
    confusion_matrices = {}
    class_labels = sorted(df['incident_type'].unique())


    #for each windows configuration
    for config_idx, config in enumerate(conf, start=0):
        print(f"\nTesting configuration {config_idx}/{len(conf)}: {config}")
        
        f1_scores = []
        y_true_global = []
        y_pred_global = []
        
        for fold, (train_index, test_index) in enumerate(skf.split(df, df['incident_type']), start=1):
            df_train, df_test = df.iloc[train_index], df.iloc[test_index]
           
                
            # Calculate event frequencies and relevance metrics
            class_event_freq, overall_event_freq = calculate_event_frequencies(df_train)
            relevance_metric = calculate_relevance_metric(class_event_freq, overall_event_freq)
            
            # Filter events based on the given threshold
            filtered_events_data_train = filter_events(relevance_metric, 0.2)

            df_train = df_train.copy()  # Create a copy of the training data
            df_test = df_test.copy()    # Create a copy of the testing data
        
            #Filter the training and the test set 
            df_train.loc[:, 'filtered_events_metric_r_train'] = df_train.apply(
                    lambda row: filter_events_row_train(row, filtered_events_data_train), axis=1
                )
                
            df_test.loc[:, 'filtered_events_metric_r_test'] = df_test.apply(
                    lambda row: filter_events_row_train(row, filtered_events_data_train), axis=1
        )
            df_test.loc[:, 'filtered_seconds_metric_r_test'] = df_test.apply(
                    lambda row: filter_second_row_train(row, filtered_events_data_train), axis=1
        )   
            

            incident_association_train = non_relevant_events_filter_r_train(
                creation_dictionary_events_related_to_seconds(df_train), windows, 12, df_train
            )
            #llCSS_var = LLCSS(df_paper, incident_association_train)
            priors_proba_trainingset = probas_priors_training_test(count_events_per_class(incident_association_train)
            )
            
            print(f"  Fold {fold}")
            
            y_true = []
            y_pred = []
           

            # We loop through the windows until the probability of a window doesn't exceed a treshold = 0.0005 (determined after many simulations)
            for incident_window, secseq,  incident_type in zip(df_test["filtered_events_metric_r_test"],df_test["filtered_seconds_metric_r_test"], df_test['incident_type']):
                if incident_window :
                
                    aggregated_posteriors = defaultdict(float)
                    i = 0
                    
                    while i < len(config) : 
                        
                        # we keep events in the corresponding window
                        incident_window_i = windows_zooming_i(incident_window, secseq,i)
                        if len(incident_window_i) ==0 : 
                            i +=1
                            continue
                        #Events hapening in only one class (in the training set) and in the window i 
                        counting_events_happening_only_in_ONE_class_var = counting_events_happening_only_in_ONE_class(
                            i, windows_xk, df_train
                        )
                        
                        #We call the Original Naives Bayes classifier
                        predicted_class, posterior_probabilities = original_naive_bayes(
                            incident_window_i, incident_association_train, priors_proba_trainingset,
                            counting_events_happening_only_in_ONE_class_var
                        )
                        
                        #The probabilities obtained for each class
                        aggregated_posteriors[i] = (predicted_class, posterior_probabilities)
                        
                        #
                        if posterior_probabilities >= threshold:
                        
                            y_pred.append(predicted_class)
                            y_true.append(incident_type)
                            print(incident_type, "true class")
                            break
                        i +=1

                        # if we don't exceed the trehsold, then we return the class with the highest probability
                    else:
                        if aggregated_posteriors:
                            best_window = max(aggregated_posteriors, key=lambda x: aggregated_posteriors[x][1])
                        
                            
                            print(incident_type, "true class")
                            y_pred.append(aggregated_posteriors[best_window][0])
                            y_true.append(incident_type)
                       
            #F1 score stuff
            y_true_global.extend(y_true)
            y_pred_global.extend(y_pred)
            
            fold_f1 = f1_score(y_true, y_pred, average="weighted")
            f1_scores.append(fold_f1)
        #  precision, recall, and F1-score for this configuration
        avg_f1 = np.mean(f1_scores)
        config_results[tuple(config)] = avg_f1
        print(avg_f1)
        conf_matrix = confusion_matrix(y_true_global, y_pred_global)
        confusion_matrices[tuple(config)] = conf_matrix
        
        print(f"\nConfusion Matrix for the configuration (window: {config}):")
        print(conf_matrix)
        print("\nClassification Report:")
        report = classification_report(y_true_global, y_pred_global,labels=class_labels,  target_names=[str(c) for c in class_labels])
        print(report)
        print(f"\nConfiguration {config}: Average F1 = {avg_f1:.4f}")
        
    print("\nFinal results by configuration:")
    for config, avg_f1 in config_results.items():
        print(f"Configuration {config}: Average F1 = {avg_f1:.4f}")
        print(f"Confusion Matrix for {config}:\n{confusion_matrices[config]}")

    return config_results, confusion_matrices

# Example call to the function
avg_f1_scores, confusion_matrices = cascaded_naive_bayes_classifier_with_configs(df_paper, conf, threshold=0.0005)



Testing configuration 0/13: [(-150.0, 150.0)]


c:\Users\matin\anaconda3\envs\sfml\Lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  warnings.warn(


  Fold 1
STAGE 1: 4048 in class 2
2 true class
STAGE 1: 1286 in class 4
4 true class
STAGE 1: 690 in class 14
14 true class
STAGE 1: 3234 in class 13
13 true class
STAGE 1: 3506 in class 2
2 true class
STAGE 1: 1772 in class 9
9 true class
13 4.490092996057174
13 true class
STAGE 1: 2972 in class 4
4 true class
STAGE 1: 2944 in class 13
13 true class
STAGE 1: 2564 in class 14
14 true class
13 0.001292759137345375
14 true class
STAGE 1: 2938 in class 13
13 true class
13 1.0841070532079524e-11
13 true class
14 0.01180675365353311
99 true class
99 1.7581796629688887e-12
99 true class
STAGE 1: 1732 in class 9
9 true class
STAGE 1: 4054 in class 2
2 true class
STAGE 1: 952 in class 13
13 true class
STAGE 1: 2942 in class 13
13 true class
STAGE 1: 2492 in class 14
14 true class
STAGE 1: 2972 in class 4
4 true class
STAGE 1: 3506 in class 2
2 true class
STAGE 1: 4076 in class 9
9 true class
STAGE 1: 4050 in class 2
2 true class
STAGE 1: 946 in class 13
13 true class
STAGE 1: 3506 in class 2
2

c:\Users\matin\anaconda3\envs\sfml\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\matin\anaconda3\envs\sfml\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\matin\anaconda3\envs\sfml\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ma

  Fold 1
STAGE 1: 4048 in class 2
2 true class
STAGE 1: 1286 in class 4
4 true class
STAGE 1: 690 in class 14
14 true class
STAGE 1: 3234 in class 13
13 true class
STAGE 1: 3506 in class 2
2 true class
STAGE 1: 1772 in class 9
9 true class
13 1.4984818981040133e-31
13 true class
13 4.490092996057174
13 true class
STAGE 1: 2972 in class 4
4 true class
STAGE 1: 2944 in class 13
13 true class
STAGE 1: 2564 in class 14
14 true class
13 0.001292759137345375
14 true class
STAGE 1: 2938 in class 13
13 true class
13 1.0841070532079524e-11
13 1.0841070532079524e-11
13 true class
13 0.20495385728533724
13 true class
14 0.01180675365353311
99 true class
99 1.7581796629688887e-12
STAGE 1: 2874 in class 99
99 true class
STAGE 1: 1732 in class 9
9 true class
STAGE 1: 4054 in class 2
2 true class
STAGE 1: 2938 in class 13
13 true class
STAGE 1: 2540 in class 14
14 true class
STAGE 1: 952 in class 13
13 true class
STAGE 1: 2942 in class 13
13 true class
STAGE 1: 2492 in class 14
14 true class
STAGE 1:

KeyboardInterrupt: 

### Cascaded Classfier : Single Approach :  we apply the 10 Fold Cross Validation on each window. 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score


def cascaded_naive_bayes_classifier_with_configs(df, conf, threshold=0.0005, n_splits=10):
    from collections import defaultdict
    from sklearn.model_selection import StratifiedKFold
    confiture = [(-150.0, 150.0), (-300.0, 300.0), (-450.0, 450.0), (-600.0, 600.0), (-900, 600), (-1200, 600)]
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    config_results = {}
    confusion_matrices = {}
    class_labels = sorted(df['incident_type'].unique())

    for config_idx, config in enumerate(confiture, start=0):
        print(f"\nTesting configuration {config_idx}/{len(conf)}: {config}")
        
        f1_scores = []
        y_true_global = []
        y_pred_global = []
        
        for fold, (train_index, test_index) in enumerate(skf.split(df, df['incident_type']), start=1):
            df_train, df_test = df.iloc[train_index], df.iloc[test_index]
            
                
            # Calculate event frequencies and relevance metrics
            class_event_freq, overall_event_freq = calculate_event_frequencies(df_train)
            relevance_metric = calculate_relevance_metric(class_event_freq, overall_event_freq)
            
            # Filter events based on the given threshold
            filtered_events_data_train = filter_events(relevance_metric, 0.2)

            df_train = df_train.copy() 
            df_test = df_test.copy()    
        
            df_train.loc[:, 'filtered_events_metric_r_train'] = df_train.apply(
                    lambda row: filter_events_row_train(row, filtered_events_data_train), axis=1
                )
                
            df_test.loc[:, 'filtered_events_metric_r_test'] = df_test.apply(
                    lambda row: filter_events_row_train(row, filtered_events_data_train), axis=1
        )
            incident_association_train = non_relevant_events_filter_r_train(
                creation_dictionary_events_related_to_seconds(df_train), windows, 12, df_train
            )
            
            priors_proba_trainingset= probas_priors_training_test(count_events_per_class((incident_association_train)
            ))

            print(f"  Fold {fold}")
            
            y_true = []
            y_pred = []
            
            for incident_window, incident_type in zip(df_test["filtered_events_metric_r_test"], df_test['incident_type']):
                if incident_window : 
                    aggregated_posteriors = defaultdict(float)
                    
                    for i, window_range in enumerate(config):
                        incident_association_train_i = non_relevant_events_filter_r_train(
                        creation_dictionary_events_related_to_seconds(df_train), windows, i, df_train
                        )
                        counting_events_happening_only_in_ONE_class_var = counting_events_happening_only_in_ONE_class(
                            i, windows_xk, df_train
                        )
                        predicted_class, posterior_probabilities = original_naive_bayes(
                            incident_window, incident_association_train_i, priors_proba_trainingset,
                            counting_events_happening_only_in_ONE_class_var
                        )
                        
                        aggregated_posteriors[i] = (predicted_class, posterior_probabilities)
                        
                        if posterior_probabilities >= threshold:
                            print("true class, " , incident_type)
                            y_pred.append(predicted_class)
                            y_true.append(incident_type)
                            break
                    else:
                        if aggregated_posteriors:
                            best_window = max(aggregated_posteriors, key=lambda x: aggregated_posteriors[x][1])
                           
                            
                            print("true class, " , incident_type)
                            y_pred.append(aggregated_posteriors[best_window][0])
                            y_true.append(incident_type)
                    
            y_true_global.extend(y_true)
            y_pred_global.extend(y_pred)
            
            fold_f1 = f1_score(y_true, y_pred, average="weighted")
            f1_scores.append(fold_f1)

        avg_f1 = np.mean(f1_scores)
        config_results[tuple(config)] = avg_f1
        print(avg_f1)
        conf_matrix = confusion_matrix(y_true_global, y_pred_global)
        confusion_matrices[tuple(config)] = conf_matrix
         
    
        print(f"\nConfusion Matrix for the first configuration (window: {config}):")
        print(conf_matrix)


        # Print the precision, recall, and F1-score for this configuration
        print("\nClassification Report:")
        report = classification_report(y_true_global, y_pred_global,labels=class_labels,  target_names=[str(c) for c in class_labels])
        print(report)
        print(f"\nConfiguration {config}: Average F1 = {avg_f1:.4f}")
        print(f"Confusion Matrix:\n{conf_matrix}")
        print(avg_f1)
       
        print(f"Classification Report for Configuration {config}:")
        print(classification_report(y_true_global, y_pred_global))

    print("\nFinal results by configuration:")
    for config, avg_f1 in config_results.items():
        print(f"Configuration {config}: Average F1 = {avg_f1:.4f}")
        print(f"Confusion Matrix for {config}:\n{confusion_matrices[config]}")

    return config_results, confusion_matrices


avg_f1_scores, confusion_matrices = cascaded_naive_bayes_classifier_with_configs(df_paper, conf, threshold=0.0005)





Testing configuration 0/13: (-150.0, 150.0)


c:\Users\matin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  warnings.warn(


  Fold 1
STAGE 1: 4048 in class 2
true class,  2
13 1.0356775099889664e-07
13 4.571194061578915e-08
true class,  14
STAGE 1: 3032 in class 4
true class,  4
STAGE 1: 690 in class 14
true class,  14
9 2.1054116738568745e-10


KeyboardInterrupt: 